In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Frankenstein MLB Pitcher Builder - Data Cleaning
The goal of this notebook is to intake pitching data from baseball savant and clean the datasets for use in our frankenstein MLB pitcher builder

Lets first create a function that will load in the dataframes for each of the different metrics we are evaulating. Each set of statistics has 3 different csv files that represent the 3 years we are evaluating on. The function below will take these 3 dataframes and will concatenate them into one dataframe that we will then clean and input into the similarity function

In [2]:
def load_csv(file):
    years = [2023, 2024, 2025]
    dfs = []
    for year in years:
        df = pd.read_csv(f'../data/{file}_{year}.csv')
        df_cols = df.columns.to_list()
        if 'year' not in df_cols:
            df['year'] = year
        dfs.append(df)

    df = pd.concat(dfs)
    return df

In [3]:
import os
os.getcwd()

'/Users/mikemichelini/Documents/mlb_frankenstein/mlb_frankenstein_player_builder/notebooks'

Lets now implement the function on all of the different datasets we will implement in this project

In [4]:
df_pitch = load_csv('percentile_rankings_pitch')
df_arm_angle = load_csv('pitcher_arm_angles')
df_zone = load_csv('swing-take-pitch')
df_pitch_usage = load_csv('pitch_arsenals_usage')
df_pitch_spin = load_csv('pitch_arsenals_spin')
df_pitch_speed = load_csv('pitch_arsenals_speed')
df_batted = load_csv('batted-ball-pitch')
df_bat_track = load_csv('bat-tracking-pitch')

Lets begin with the percentiles dataframe. This dataframe contains the percentile ranking of each player in several different key pitching categories such as whiff percent, xwoba and xera. We will print the dataframe to see what we are dealing with

In [5]:
df_pitch

,player_name,player_id,year,xwoba,xba,xslg,xiso,xobp,brl,brl_percent,...,hard_hit_percent,k_percent,bb_percent,whiff_percent,chase_percent,arm_strength,xera,fb_velocity,fb_spin,curve_spin
0,"Lucchesi, Joey",664192,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,35.0,7.0
1,"Edwards Jr., Carl",605218,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.0,99.0,14.0
2,"Long, Sam",669674,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.0,58.0,63.0
3,"Patiño, Luis",672715,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.0,41.0,NaN
4,"Tarnok, Freddy",676206,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71.0,94.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706,"Curtiss, John",595928,2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.0,67.0,NaN
707,"Hudson, Bryan",663542,2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,27.0,NaN
708,"Enns, Dietrich",608650,2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.0,13.0,NaN
709,"Horn, Bailey",690544,2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67.0,71.0,NaN


In [6]:
df_pitch.columns

Index(['player_name', 'player_id', 'year', 'xwoba', 'xba', 'xslg', 'xiso',
       'xobp', 'brl', 'brl_percent', 'exit_velocity', 'max_ev',
       'hard_hit_percent', 'k_percent', 'bb_percent', 'whiff_percent',
       'chase_percent', 'arm_strength', 'xera', 'fb_velocity', 'fb_spin',
       'curve_spin'],
      dtype='object')

Ok so there are a few columns we do not need because they will be covered in other dataframes so lets go ahead and drop those columns

In [7]:
df_pitch = df_pitch.drop(columns = ['arm_strength','fb_velocity', 'fb_spin', 'curve_spin', 'exit_velocity', 'max_ev'])

Ok lets see how many NA values there are now

In [8]:
df_pitch.isna().sum()

player_name            0
player_id              0
year                   0
xwoba               1027
xba                 1027
xslg                1027
xiso                1027
xobp                1027
brl                 1027
brl_percent         1027
hard_hit_percent    1027
k_percent           1027
bb_percent          1027
whiff_percent       1027
chase_percent       1027
xera                1027
dtype: int64

We can see that there are a bunch of rows with the same number of NA values which tells me that the same players have all of these values missing from the dataset. If this is the case then we will likely drop them from the dataset because almost all of their statistical data is missing and it wouldnt make any sense to try and impute these values as it wouldnt represent the player well

In [9]:
df_pitch[df_pitch['xwoba'].isna()].head(25)

,player_name,player_id,year,xwoba,xba,xslg,xiso,xobp,brl,brl_percent,hard_hit_percent,k_percent,bb_percent,whiff_percent,chase_percent,xera
0,"Lucchesi, Joey",664192,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Edwards Jr., Carl",605218,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Long, Sam",669674,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Patiño, Luis",672715,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Tarnok, Freddy",676206,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"Wantz, Andrew",681806,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"Garcia, Luis",677651,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,"Castillo, Diego",650895,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,"Richardson, Lyon",680689,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,"Dubin, Shawn",681869,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


So as expected these guys all have missing values in the same columns so we are going to delete any rows in which all of these values are NA

In [10]:
cols = [x for x in df_pitch if x not in ['player_name', 'player_id', 'year', 'max_ev']]
df_pitch = df_pitch.dropna(how = 'all', subset = cols)

Great lets check how many NA values we have now. I would assume that we now have zero if my original assumption was correct

In [11]:
df_pitch.isna().sum()

player_name         0
player_id           0
year                0
xwoba               0
xba                 0
xslg                0
xiso                0
xobp                0
brl                 0
brl_percent         0
hard_hit_percent    0
k_percent           0
bb_percent          0
whiff_percent       0
chase_percent       0
xera                0
dtype: int64

Great now that this dataframe is all cleaned up lets move onto the next dataset which is the arm angle dataset. This dataset will tell us the pitching hand of each pitcher in addition to the release angle of their arm as they deliver the pitch. This can help us to group together submarine pitchers who have an extremely low release angle and pitchers who throw more over the top with a high release angle. We will start the same way by viewing the dataframe and dropping any unneeded columns

In [12]:
df_arm_angle

,pitcher,pitcher_name,pitch_hand,n_pitches,team_id,ball_angle,relative_release_ball_x,release_ball_z,relative_shoulder_x,shoulder_z,year
0,425794,"Wainwright, Adam",R,1680,138,45.5,-1.646190,6.228780,-0.077938,4.633131,2023
1,425844,"Greinke, Zack",R,2218,118,43.8,-1.587213,6.209671,-0.029719,4.717702,2023
2,434378,"Verlander, Justin",R,2602,9999,55.1,-1.259901,6.994350,0.150379,4.965253,2023
3,446372,"Kluber, Corey",R,981,111,31.4,-2.369732,5.033456,-0.512560,3.900661,2023
4,448179,"Hill, Rich",L,2497,9999,36.8,2.310753,5.621806,0.451061,4.219176,2023
...,...,...,...,...,...,...,...,...,...,...,...
282,702352,"Bivens, Spencer",R,1294,137,36.8,-2.085303,6.198733,-0.224609,4.808545,2025
283,801403,"Dollander, Chase",R,1778,115,25.8,-2.400959,5.388937,-0.260844,4.356332,2025
284,805673,"Matthews, Zebby",R,1352,142,39.6,-1.929234,5.876376,-0.267775,4.505112,2025
285,806185,"Birdsong, Hayden",R,1191,137,48.2,-1.206869,6.475953,0.335368,4.751983,2025


Ok so as mentioned above we only need a few of these columns from the dataset so we will filter to just this small subset. The other columns dive into much further depth regarding the release point of each pitcher which is beyond the scope of this analysis

In [13]:
df_arm_angle = df_arm_angle[['pitcher', 'pitcher_name', 'pitch_hand', 'ball_angle', 'year']]

Great lets now take a look at the NA values to see how we can handle them

In [14]:
df_arm_angle.isna().sum()

pitcher         0
pitcher_name    0
pitch_hand      0
ball_angle      0
year            0
dtype: int64

Awesome we do not have any NA values so lets go ahead and merge this in with the percentile dataframe. We will merge the dataframes on both player ID and year because there are typically several players in the MLB that have very similar or the exact same names. Merging on player ID helps us mitigate players with the same name interfering with the merge. We will also include an indicator column which will tell us how many rows did not merge. Since df_pitch contains most of the values we will use in this analysis, we will use the how = 'left' method because in the scenario where there is data for a player in the df_pitch dataframe but not a separate dataframe, we want to be able to keep that player in the analysis instead of deleting them by implementing an inner merge. Finally when we merge, we will first merge on player ID and then by year since we are using 3 years worth of data

In [15]:
df_pitch_arm_angle = df_pitch.merge(df_arm_angle, left_on = ['player_id', 'year'], right_on = ['pitcher', 'year'],
                                    how = 'left', indicator = True)

Lets see how many of the values did not merge together and see how we can deal with these missing values

In [16]:
df_pitch_arm_angle['_merge'].value_counts()

_merge
both          853
left_only     234
right_only      0
Name: count, dtype: int64

Ok so we have a decent amount of values that did not merge together between the two so lets find out as to why by filtering to only the values that did not merge

In [17]:
df_pitch_arm_angle[df_pitch_arm_angle['_merge']!='both']

,player_name,player_id,year,xwoba,xba,xslg,xiso,xobp,brl,brl_percent,...,k_percent,bb_percent,whiff_percent,chase_percent,xera,pitcher,pitcher_name,pitch_hand,ball_angle,_merge
8,"Poche, Colin",621363,2023,98.0,96.0,96.0,91.0,94.0,83.0,83.0,...,60.0,29.0,71.0,78.0,98.0,NaN,NaN,NaN,NaN,left_only
13,"Lauer, Eric",641778,2023,1.0,1.0,1.0,1.0,2.0,44.0,1.0,...,30.0,12.0,20.0,14.0,1.0,NaN,NaN,NaN,NaN,left_only
18,"Moreta, Dauri",664294,2023,94.0,98.0,88.0,67.0,93.0,74.0,46.0,...,95.0,26.0,92.0,6.0,94.0,NaN,NaN,NaN,NaN,left_only
20,"Schreiber, John",670167,2023,53.0,75.0,75.0,71.0,25.0,87.0,73.0,...,71.0,7.0,38.0,17.0,53.0,NaN,NaN,NaN,NaN,left_only
23,"Rogers, Taylor",573124,2023,76.0,88.0,79.0,66.0,57.0,68.0,22.0,...,89.0,11.0,56.0,35.0,76.0,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1065,"Mears, Nick",683232,2025,30.0,20.0,11.0,11.0,59.0,55.0,14.0,...,36.0,79.0,49.0,93.0,30.0,NaN,NaN,NaN,NaN,left_only
1073,"Pomeranz, Drew",519141,2025,73.0,80.0,71.0,61.0,72.0,81.0,44.0,...,84.0,63.0,44.0,40.0,73.0,NaN,NaN,NaN,NaN,left_only
1078,"Ureña, José",570632,2025,1.0,2.0,1.0,1.0,1.0,39.0,1.0,...,2.0,26.0,11.0,17.0,1.0,NaN,NaN,NaN,NaN,left_only
1079,"Hader, Josh",623352,2025,100.0,100.0,100.0,85.0,100.0,89.0,53.0,...,99.0,53.0,99.0,99.0,100.0,NaN,NaN,NaN,NaN,left_only


It seems like most of these guys are actually relief pitchers, but arm angle is not something we can totally assume the median on because there are some submarine pitchers in the league who have an arm angle that is significantly different from the rest. Lets first try to forward and back fill the arm angle and pitch hand because we may have years in which we have this data for specific players and some years where we dont. Arm angle is something I would not expect to change a bunch from year to year so this method should be a reasonable assumption.

In [18]:
df_pitch_arm_angle[['pitch_hand', 'ball_angle']] = (
    df_pitch_arm_angle
      .groupby("player_id")[['pitch_hand', 'ball_angle']]
      .transform(lambda x: x.ffill().bfill())
)

/var/folders/d8/29ff9cz17g5blxpv2039mrj80000gn/T/ipykernel_16865/702849554.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .transform(lambda x: x.ffill().bfill())


Lets check how many values this filled by checking how many NA values remain with the pitcher hand and arm angle columns

In [19]:
df_pitch_arm_angle.isna().sum()

player_name           0
player_id             0
year                  0
xwoba                 0
xba                   0
xslg                  0
xiso                  0
xobp                  0
brl                   0
brl_percent           0
hard_hit_percent      0
k_percent             0
bb_percent            0
whiff_percent         0
chase_percent         0
xera                  0
pitcher             234
pitcher_name        234
pitch_hand          133
ball_angle          133
_merge                0
dtype: int64

Ok so it looks like we were able to fill in around 100 of the values that were missing. Lets go ahead and drop the columns we do not need any longer from the merge.

In [20]:
df_pitch_arm_angle = df_pitch_arm_angle.drop(columns = ['pitcher', 'pitcher_name', '_merge'])

We will look at the NA values again to confirm

In [21]:
df_pitch_arm_angle.isna().sum()

player_name           0
player_id             0
year                  0
xwoba                 0
xba                   0
xslg                  0
xiso                  0
xobp                  0
brl                   0
brl_percent           0
hard_hit_percent      0
k_percent             0
bb_percent            0
whiff_percent         0
chase_percent         0
xera                  0
pitch_hand          133
ball_angle          133
dtype: int64

We are likely going to have to do a manual fix for the pitcher hand and may assume the median for the ball angle but lets move on to the other datasets for now. We will now work with the df_zone dataset. This dataset will tell us how each pitcher performs on their pitches in different parts of the zone. For example, runs_heart tells us how the pitcher performs on pitches they throw right down the middle of the zone while runs_shadow tells us how the pitcher performs on pitches on the edges of the zone. Lets view this dataframe to see the columns we have

In [22]:
df_zone

,year,"last_name, first_name",player_id,team_id,pa,pitches,runs_all,runs_heart,runs_shadow,runs_chase,runs_waste
0,2023,"Moll, Sam",594580,113,260,1025,9.040052,7.437342,12.857906,-4.711863,-6.543333
1,2023,"Wentz, Joey",666214,116,492,1930,-25.185273,-8.127591,6.051475,-16.094408,-7.014749
2,2023,"Clarke, Taylor",664199,118,265,1068,-11.437555,-3.648077,2.674606,-7.158609,-3.305474
3,2023,"Gray, Sonny",543243,142,754,2818,36.558660,31.553201,24.966486,-12.053655,-7.907371
4,2023,"Ryan, Joe",657746,142,671,2683,-0.143532,8.872141,11.718316,-12.249044,-8.484945
...,...,...,...,...,...,...,...,...,...,...,...
295,2025,"Kerkering, Orion",689147,143,263,1073,-1.859740,0.690174,6.998202,-5.975504,-3.572612
296,2025,"Muñoz, Andrés",662253,136,251,1035,16.194942,12.046152,11.438117,-2.354216,-4.935111
297,2025,"Gilbert, Logan",669302,136,535,2156,9.495711,-1.257652,20.677396,-1.740721,-8.183312
298,2025,"Milner, Hoby",571948,140,290,1157,3.421657,10.401867,5.446770,-9.978690,-2.448289


To ensure we are not double counting the runs here, we are going to drop the runs_all column. We will also drop the pitches, pa and team_id columns as they are not relevant to this analysis. 

In [23]:
df_zone = df_zone.drop(columns = ['team_id', 'pa', 'pitches', 'runs_all'])

Great lets check how many NA values there are now with this dataset

In [24]:
df_zone.isna().sum()

year                     0
last_name, first_name    0
player_id                0
runs_heart               0
runs_shadow              0
runs_chase               0
runs_waste               0
dtype: int64

Awesome we can see there are no NA values in this dataset so lets merge the values in with the compiled dataframe so far

In [25]:
df_pitch_arm_angle_zone = df_pitch_arm_angle.merge(df_zone, left_on = ['player_id', 'year'], right_on = ['player_id', 'year'],
                                                   how = 'left', indicator = True)

Lets see how many of these values merged in by checking the merge column

In [26]:
df_pitch_arm_angle_zone['_merge'].value_counts()

_merge
both          900
left_only     187
right_only      0
Name: count, dtype: int64

Ok so there are a good amount of players that did not merge in so lets check their values by filtering for only those values that did not merge

In [27]:
df_rp = df_pitch_arm_angle_zone[df_pitch_arm_angle_zone['_merge']!='both']

In [28]:
df_rp

,player_name,player_id,year,xwoba,xba,xslg,xiso,xobp,brl,brl_percent,...,chase_percent,xera,pitch_hand,ball_angle,"last_name, first_name",runs_heart,runs_shadow,runs_chase,runs_waste,_merge
8,"Poche, Colin",621363,2023,98.0,96.0,96.0,91.0,94.0,83.0,83.0,...,78.0,98.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
13,"Lauer, Eric",641778,2023,1.0,1.0,1.0,1.0,2.0,44.0,1.0,...,14.0,1.0,L,38.6,NaN,NaN,NaN,NaN,NaN,left_only
20,"Schreiber, John",670167,2023,53.0,75.0,75.0,71.0,25.0,87.0,73.0,...,17.0,53.0,R,11.8,NaN,NaN,NaN,NaN,NaN,left_only
23,"Rogers, Taylor",573124,2023,76.0,88.0,79.0,66.0,57.0,68.0,22.0,...,35.0,76.0,L,29.1,NaN,NaN,NaN,NaN,NaN,left_only
33,"Nelson, Kyle",669459,2023,69.0,65.0,40.0,30.0,89.0,60.0,23.0,...,96.0,69.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1064,"Kittredge, Andrew",552640,2025,94.0,83.0,79.0,70.0,97.0,89.0,75.0,...,100.0,94.0,R,39.2,NaN,NaN,NaN,NaN,NaN,left_only
1065,"Mears, Nick",683232,2025,30.0,20.0,11.0,11.0,59.0,55.0,14.0,...,93.0,30.0,R,60.8,NaN,NaN,NaN,NaN,NaN,left_only
1073,"Pomeranz, Drew",519141,2025,73.0,80.0,71.0,61.0,72.0,81.0,44.0,...,40.0,73.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1078,"Ureña, José",570632,2025,1.0,2.0,1.0,1.0,1.0,39.0,1.0,...,17.0,1.0,R,31.3,NaN,NaN,NaN,NaN,NaN,left_only


It looks like to me that most if not all these players are relief pitchers who simply didnt have enough innings to qualify. For now we are going to leave these values as NA because pitching performance changes from year to year so we cannot just assume they will be same from one year to the next by forward or backfilling values. In addition it is difficult to impute values because we do not want to impute values for a starting pitcher into a relief pitcher. Lets go ahead and leave these values as NA for now and drop the columns we do not need anymore

In [29]:
df_pitch_arm_angle_zone = df_pitch_arm_angle_zone.drop(columns = ['last_name, first_name', '_merge'])

Ok lets move on to the next dataframe which is the pitch usage dataframe. This dataframe will tell us the percentage of time each pitcher throws each of the pitches in their arsenal. For example, n_ff is the percentage of time a pitcher throws their fastball

In [30]:
df_pitch_usage

,"last_name, first_name",pitcher,n_ff,n_si,n_fc,n_sl,n_ch,n_cu,n_fs,n_kn,n_st,n_sv,year
0,"Cole, Gerrit",543037,52.9,0.0,7.0,20.8,7.1,12.1,NaN,NaN,NaN,NaN,2023
1,"Cease, Dylan",656302,43.2,NaN,NaN,38.6,3.0,15.2,NaN,NaN,NaN,NaN,2023
2,"Gallen, Zac",668678,49.7,0.0,9.2,4.5,13.9,22.7,NaN,NaN,NaN,NaN,2023
3,"Castillo, Luis",622491,44.3,18.0,NaN,22.1,15.6,NaN,NaN,NaN,NaN,NaN,2023
4,"Mikolas, Miles",571945,26.0,24.4,NaN,24.3,5.4,20.0,NaN,NaN,NaN,NaN,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,"Morales, Luis",806960,52.4,NaN,NaN,8.0,13.0,NaN,NaN,NaN,26.6,NaN,2025
340,"Beck, Tristan",663941,33.0,NaN,NaN,21.3,NaN,17.3,NaN,NaN,28.4,NaN,2025
341,"Festa, Matt",670036,39.2,8.1,14.6,NaN,NaN,NaN,NaN,NaN,38.2,NaN,2025
342,"Wilson, Justin",458677,46.8,NaN,14.8,33.3,NaN,NaN,5.0,NaN,NaN,NaN,2025


We are going to have a lot of expected NA values in this dataframe because not every pitcher throws the same exact pitches, so we will set all of the NA values to zero in this dataset to reflect that these pitchers do not throw that specific type of pitch

In [31]:
df_pitch_usage[['n_ff', 'n_si', 'n_fc', 'n_sl', 'n_ch', 'n_cu', 'n_fs', 'n_kn',
                'n_st', 'n_sv']] = df_pitch_usage[['n_ff', 'n_si', 'n_fc', 'n_sl', 'n_ch', 'n_cu', 'n_fs', 'n_kn',
                'n_st', 'n_sv']].fillna(0)

Lets check to see that line of code worked as expected by checking the number of NA values

In [32]:
df_pitch_usage.isna().sum()

last_name, first_name    0
pitcher                  0
n_ff                     0
n_si                     0
n_fc                     0
n_sl                     0
n_ch                     0
n_cu                     0
n_fs                     0
n_kn                     0
n_st                     0
n_sv                     0
year                     0
dtype: int64

Great there are no more NA values in this dataset so lets go ahead and merge it with the compiled dataframe so far

In [33]:
df_pitch_arm_angle_zone_usage = df_pitch_arm_angle_zone.merge(df_pitch_usage, left_on = ['player_id', 'year'],
                                                              right_on = ['pitcher', 'year'], how = 'left', indicator = True)

Lets check to see which of the values did not merge in the dataet

In [34]:
df_pitch_arm_angle_zone_usage['_merge'].value_counts()

_merge
both          1060
left_only       27
right_only       0
Name: count, dtype: int64

Ok so only 27 this time did not merge over, but lets take a look at who they are to see if we can deduce anything

In [35]:
df_pitch_arm_angle_zone_usage[df_pitch_arm_angle_zone_usage['_merge']!='both']

,player_name,player_id,year,xwoba,xba,xslg,xiso,xobp,brl,brl_percent,...,n_si,n_fc,n_sl,n_ch,n_cu,n_fs,n_kn,n_st,n_sv,_merge
102,"Machado, Andrés",600921,2023,9.0,7.0,3.0,3.0,29.0,52.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
126,"Lynch IV, Daniel",663738,2023,21.0,17.0,16.0,19.0,42.0,65.0,54.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
162,"May, Trevor",543507,2023,33.0,47.0,62.0,70.0,4.0,87.0,81.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
219,"Mayza, Tim",641835,2023,82.0,35.0,81.0,92.0,64.0,95.0,94.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
234,"Ureña, José",570632,2023,6.0,17.0,5.0,2.0,12.0,56.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
236,"Armstrong, Shawn",542888,2023,98.0,88.0,92.0,89.0,98.0,97.0,95.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
292,"Fleming, Josh",676596,2023,11.0,3.0,16.0,39.0,10.0,79.0,83.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
315,"Alexander, Scott",518397,2023,72.0,19.0,69.0,88.0,58.0,92.0,94.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
337,"Almonte, Yency",622075,2023,41.0,39.0,68.0,77.0,14.0,83.0,68.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
357,"Speier, Gabe",642100,2023,80.0,52.0,63.0,70.0,79.0,68.0,34.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


I am not entirely sure why these values are NA. Again it looks like it is mostly relief pitchers but several of these pitchers should have data from previous years that we could backfill or forward fill. Pitch usage typically doesnt change much from year to year with these pitchers so I think these values would provide a reasonable estimate if we were to forward or backfill across years

In [36]:
df_pitch_arm_angle_zone_usage[['n_ff', 'n_si', 'n_fc', 'n_sl', 'n_ch', 'n_cu', 'n_fs', 'n_kn',
                'n_st', 'n_sv']] = (
    df_pitch_arm_angle_zone_usage
      .groupby("player_id")[['n_ff', 'n_si', 'n_fc', 'n_sl', 'n_ch', 'n_cu', 'n_fs', 'n_kn',
                'n_st', 'n_sv']]
      .transform(lambda x: x.ffill().bfill())
)

Lets check to see if this changed any of these pitchers values by viewing the values that didnt merge again

In [37]:
df_pitch_arm_angle_zone_usage[df_pitch_arm_angle_zone_usage['_merge']!='both']

,player_name,player_id,year,xwoba,xba,xslg,xiso,xobp,brl,brl_percent,...,n_si,n_fc,n_sl,n_ch,n_cu,n_fs,n_kn,n_st,n_sv,_merge
102,"Machado, Andrés",600921,2023,9.0,7.0,3.0,3.0,29.0,52.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
126,"Lynch IV, Daniel",663738,2023,21.0,17.0,16.0,19.0,42.0,65.0,54.0,...,19.0,0.0,29.2,19.2,9.8,0.0,0.0,0.0,0.0,left_only
162,"May, Trevor",543507,2023,33.0,47.0,62.0,70.0,4.0,87.0,81.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
219,"Mayza, Tim",641835,2023,82.0,35.0,81.0,92.0,64.0,95.0,94.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
234,"Ureña, José",570632,2023,6.0,17.0,5.0,2.0,12.0,56.0,12.0,...,41.9,0.0,34.6,18.2,0.0,0.0,0.0,0.0,0.0,left_only
236,"Armstrong, Shawn",542888,2023,98.0,88.0,92.0,89.0,98.0,97.0,95.0,...,27.1,31.0,0.0,0.0,0.0,0.0,0.0,5.7,3.0,left_only
292,"Fleming, Josh",676596,2023,11.0,3.0,16.0,39.0,10.0,79.0,83.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
315,"Alexander, Scott",518397,2023,72.0,19.0,69.0,88.0,58.0,92.0,94.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
337,"Almonte, Yency",622075,2023,41.0,39.0,68.0,77.0,14.0,83.0,68.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
357,"Speier, Gabe",642100,2023,80.0,52.0,63.0,70.0,79.0,68.0,34.0,...,29.5,0.0,25.7,0.0,0.0,0.0,0.0,0.0,0.0,left_only


Ok so it looks like we were able to fill in a few of the pitchers that did not merge. We may have to drop the others from the dataset who have no information on their pitch usage as these columns are important in determining the similarity between pitchers. Before we do that lets drop any columns from the merge that we no longer need

In [38]:
df_pitch_arm_angle_zone_usage = df_pitch_arm_angle_zone_usage.drop(columns = ['last_name, first_name', 
                                                                              'pitcher', '_merge'])

Great lets move on to the next dataset which is the pitch spin dataframe. This dataset gives the average spin in revolutions per minute for each pitch in a pitchers arsenal. For example, ff_avg_spin gives the average spin of a pitchers fastball.  I would expect this dataset to be similar in nature with the NA values to the pitch usage dataframe. Lets take a look at a preview of the dataframe

In [39]:
df_pitch_spin

,"last_name, first_name",pitcher,ff_avg_spin,si_avg_spin,fc_avg_spin,sl_avg_spin,ch_avg_spin,cu_avg_spin,fs_avg_spin,kn_avg_spin,st_avg_spin,sv_avg_spin,year
0,"Cole, Gerrit",543037,2412.0,2249.0,2510.0,2542.0,1581.0,2783.0,NaN,NaN,NaN,NaN,2023
1,"Cease, Dylan",656302,2519.0,NaN,NaN,2795.0,1678.0,2641.0,NaN,NaN,NaN,NaN,2023
2,"Gallen, Zac",668678,2297.0,2394.0,2334.0,2427.0,1517.0,2463.0,NaN,NaN,NaN,NaN,2023
3,"Castillo, Luis",622491,2287.0,2099.0,NaN,2347.0,1906.0,NaN,NaN,NaN,NaN,NaN,2023
4,"Mikolas, Miles",571945,2276.0,2192.0,NaN,2335.0,1640.0,2490.0,NaN,NaN,NaN,NaN,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,"Morales, Luis",806960,2520.0,NaN,NaN,2879.0,2075.0,NaN,NaN,NaN,2999.0,NaN,2025
340,"Beck, Tristan",663941,2130.0,NaN,NaN,2486.0,NaN,2364.0,NaN,NaN,2808.0,NaN,2025
341,"Festa, Matt",670036,2512.0,2436.0,2488.0,NaN,NaN,NaN,NaN,NaN,2666.0,NaN,2025
342,"Wilson, Justin",458677,2252.0,NaN,2371.0,2290.0,NaN,NaN,1130.0,NaN,NaN,NaN,2025


As suspected there are several missing values to this dataset because not every pitcher throws every pitch in the dataset, so we will fill all NA values with zero to represent pitchers that do not throw any specific pitch

Lets also check to make sure that there are no rows where all values are NA

In [40]:
spin_cols = [
    'ff_avg_spin', 'si_avg_spin', 'fc_avg_spin', 'sl_avg_spin',
    'ch_avg_spin', 'cu_avg_spin', 'fs_avg_spin', 'kn_avg_spin',
    'st_avg_spin', 'sv_avg_spin'
]

mask_all_na = df_pitch_spin[spin_cols].isna().all(axis=1)

df_pitch_spin[mask_all_na]

,"last_name, first_name",pitcher,ff_avg_spin,si_avg_spin,fc_avg_spin,sl_avg_spin,ch_avg_spin,cu_avg_spin,fs_avg_spin,kn_avg_spin,st_avg_spin,sv_avg_spin,year


Ok lets go ahead and fill all NA values with 0 to represent the pitches that pitchers do not throw

In [41]:
df_pitch_spin[['ff_avg_spin', 'si_avg_spin', 'fc_avg_spin', 'sl_avg_spin', 'ch_avg_spin', 'cu_avg_spin',
               'fs_avg_spin', 'kn_avg_spin', 'st_avg_spin', 'sv_avg_spin']] = df_pitch_spin[
                   ['ff_avg_spin', 'si_avg_spin', 'fc_avg_spin', 'sl_avg_spin', 'ch_avg_spin', 'cu_avg_spin',
               'fs_avg_spin', 'kn_avg_spin', 'st_avg_spin', 'sv_avg_spin']].fillna(0)

Great lets check the NA values for each column to see if this filled in all of our NA values

In [42]:
df_pitch_spin.isna().sum()

last_name, first_name    0
pitcher                  0
ff_avg_spin              0
si_avg_spin              0
fc_avg_spin              0
sl_avg_spin              0
ch_avg_spin              0
cu_avg_spin              0
fs_avg_spin              0
kn_avg_spin              0
st_avg_spin              0
sv_avg_spin              0
year                     0
dtype: int64

This looks to have filled in all of our values so lets proceed with the merge on the compiled dataframe so far

In [43]:
df_pitch_arm_angle_zone_usage_spin = df_pitch_arm_angle_zone_usage.merge(df_pitch_spin, left_on = ['player_id', 'year'],
                                                                         right_on = ['pitcher', 'year'], how = 'left',
                                                                         indicator = True)

Lets check to see if all of the values merged

In [44]:
df_pitch_arm_angle_zone_usage_spin['_merge'].value_counts()

_merge
both          1060
left_only       27
right_only       0
Name: count, dtype: int64

As we expected there are the same exact number of values that did not merge over so lets try to forward and backfill the values that are missing to reduce the number of NA values because again I would expect that pitchers have relatively the same spin from year to year on their pitches

In [45]:
df_pitch_arm_angle_zone_usage_spin[df_pitch_arm_angle_zone_usage_spin['_merge']!='both']

,player_name,player_id,year,xwoba,xba,xslg,xiso,xobp,brl,brl_percent,...,si_avg_spin,fc_avg_spin,sl_avg_spin,ch_avg_spin,cu_avg_spin,fs_avg_spin,kn_avg_spin,st_avg_spin,sv_avg_spin,_merge
102,"Machado, Andrés",600921,2023,9.0,7.0,3.0,3.0,29.0,52.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
126,"Lynch IV, Daniel",663738,2023,21.0,17.0,16.0,19.0,42.0,65.0,54.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
162,"May, Trevor",543507,2023,33.0,47.0,62.0,70.0,4.0,87.0,81.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
219,"Mayza, Tim",641835,2023,82.0,35.0,81.0,92.0,64.0,95.0,94.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
234,"Ureña, José",570632,2023,6.0,17.0,5.0,2.0,12.0,56.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
236,"Armstrong, Shawn",542888,2023,98.0,88.0,92.0,89.0,98.0,97.0,95.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
292,"Fleming, Josh",676596,2023,11.0,3.0,16.0,39.0,10.0,79.0,83.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
315,"Alexander, Scott",518397,2023,72.0,19.0,69.0,88.0,58.0,92.0,94.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
337,"Almonte, Yency",622075,2023,41.0,39.0,68.0,77.0,14.0,83.0,68.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
357,"Speier, Gabe",642100,2023,80.0,52.0,63.0,70.0,79.0,68.0,34.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


It seems to me that these are the same 27 pitchers that were NA in the previous dataset too so we will back and forward fill

In [46]:
df_pitch_arm_angle_zone_usage_spin[['ff_avg_spin', 'si_avg_spin', 'fc_avg_spin', 'sl_avg_spin', 'ch_avg_spin', 'cu_avg_spin',
               'fs_avg_spin', 'kn_avg_spin', 'st_avg_spin', 'sv_avg_spin']] = (
    df_pitch_arm_angle_zone_usage_spin
      .groupby("player_id")[['ff_avg_spin', 'si_avg_spin', 'fc_avg_spin', 'sl_avg_spin', 'ch_avg_spin', 'cu_avg_spin',
               'fs_avg_spin', 'kn_avg_spin', 'st_avg_spin', 'sv_avg_spin']]
      .transform(lambda x: x.ffill().bfill())
)

Lets check to confirm that some values were filled in as a result of this

In [47]:
df_pitch_arm_angle_zone_usage_spin[df_pitch_arm_angle_zone_usage_spin['_merge']!='both']

,player_name,player_id,year,xwoba,xba,xslg,xiso,xobp,brl,brl_percent,...,si_avg_spin,fc_avg_spin,sl_avg_spin,ch_avg_spin,cu_avg_spin,fs_avg_spin,kn_avg_spin,st_avg_spin,sv_avg_spin,_merge
102,"Machado, Andrés",600921,2023,9.0,7.0,3.0,3.0,29.0,52.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
126,"Lynch IV, Daniel",663738,2023,21.0,17.0,16.0,19.0,42.0,65.0,54.0,...,2093.0,0.0,2290.0,1642.0,2257.0,0.0,0.0,0.0,0.0,left_only
162,"May, Trevor",543507,2023,33.0,47.0,62.0,70.0,4.0,87.0,81.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
219,"Mayza, Tim",641835,2023,82.0,35.0,81.0,92.0,64.0,95.0,94.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
234,"Ureña, José",570632,2023,6.0,17.0,5.0,2.0,12.0,56.0,12.0,...,2140.0,0.0,2286.0,1801.0,0.0,0.0,0.0,0.0,0.0,left_only
236,"Armstrong, Shawn",542888,2023,98.0,88.0,92.0,89.0,98.0,97.0,95.0,...,2124.0,2463.0,0.0,0.0,0.0,0.0,0.0,2597.0,2634.0,left_only
292,"Fleming, Josh",676596,2023,11.0,3.0,16.0,39.0,10.0,79.0,83.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
315,"Alexander, Scott",518397,2023,72.0,19.0,69.0,88.0,58.0,92.0,94.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
337,"Almonte, Yency",622075,2023,41.0,39.0,68.0,77.0,14.0,83.0,68.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
357,"Speier, Gabe",642100,2023,80.0,52.0,63.0,70.0,79.0,68.0,34.0,...,2088.0,0.0,2185.0,0.0,0.0,0.0,0.0,0.0,0.0,left_only


Great we can see that several of them indeed were filled in so lets drop the remaining columns from the merge that we no longer need

In [48]:
df_pitch_arm_angle_zone_usage_spin = df_pitch_arm_angle_zone_usage_spin.drop(columns = ['last_name, first_name',
                                                                                        'pitcher', '_merge'])

Lets now move on to the next dataset which is the pitch speed dataset. This again is similar to the previous two datasets but instead this time it gives the average speed of each pitch in a pitcher's arsenal in miles per hour. I also would assume we are going to have a similar number of NA values as before.

In [49]:
df_pitch_speed

,"last_name, first_name",pitcher,ff_avg_speed,si_avg_speed,fc_avg_speed,sl_avg_speed,ch_avg_speed,cu_avg_speed,fs_avg_speed,kn_avg_speed,st_avg_speed,sv_avg_speed,year
0,"Cole, Gerrit",543037,96.7,95.4,92.7,89.1,89.1,82.9,NaN,NaN,NaN,NaN,2023
1,"Cease, Dylan",656302,95.6,NaN,NaN,86.3,74.6,80.2,NaN,NaN,NaN,NaN,2023
2,"Gallen, Zac",668678,93.5,93.8,90.2,87.4,86.8,82.5,NaN,NaN,NaN,NaN,2023
3,"Castillo, Luis",622491,96.3,95.9,NaN,85.7,88.3,NaN,NaN,NaN,NaN,NaN,2023
4,"Mikolas, Miles",571945,93.3,92.7,NaN,86.4,84.4,75.8,NaN,NaN,NaN,NaN,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,"Morales, Luis",806960,97.3,NaN,NaN,85.8,89.8,NaN,NaN,NaN,82.3,NaN,2025
340,"Beck, Tristan",663941,94.7,NaN,NaN,89.8,NaN,80.6,NaN,NaN,83.1,NaN,2025
341,"Festa, Matt",670036,92.1,93.1,90.7,NaN,NaN,NaN,NaN,NaN,83.1,NaN,2025
342,"Wilson, Justin",458677,94.5,NaN,91.4,88.0,NaN,NaN,85.1,NaN,NaN,NaN,2025


Lets fill in the NA values wtih zero to represent that these pitchers dont throw every pitch

In [50]:
df_pitch_speed[['ff_avg_speed', 'si_avg_speed', 'fc_avg_speed', 'sl_avg_speed', 'ch_avg_speed', 'cu_avg_speed',
               'fs_avg_speed', 'kn_avg_speed', 'st_avg_speed', 'sv_avg_speed']] = df_pitch_speed[
                   ['ff_avg_speed', 'si_avg_speed', 'fc_avg_speed', 'sl_avg_speed', 'ch_avg_speed', 'cu_avg_speed',
               'fs_avg_speed', 'kn_avg_speed', 'st_avg_speed', 'sv_avg_speed']].fillna(0)

Lets check to see that all of the NA values are gone from the dataset

In [51]:
df_pitch_speed.isna().sum()

last_name, first_name    0
pitcher                  0
ff_avg_speed             0
si_avg_speed             0
fc_avg_speed             0
sl_avg_speed             0
ch_avg_speed             0
cu_avg_speed             0
fs_avg_speed             0
kn_avg_speed             0
st_avg_speed             0
sv_avg_speed             0
year                     0
dtype: int64

Great lets complete the merge with the compiled set now

In [52]:
df_pitch_arm_angle_zone_usage_spin_speed = df_pitch_arm_angle_zone_usage_spin.merge(df_pitch_speed, left_on = ['player_id', 'year'],
                                                                                    right_on = ['pitcher', 'year'], how = 'left',
                                                                                    indicator = True)

Ok lets check that everything did indeed merge together

In [53]:
df_pitch_arm_angle_zone_usage_spin_speed['_merge'].value_counts()

_merge
both          1060
left_only       27
right_only       0
Name: count, dtype: int64

As expected the same pitchers are the ones missing in this dataset as well so lets do the standard forward and backfill to try and delete some of these missing values

In [54]:
df_pitch_arm_angle_zone_usage_spin_speed[['ff_avg_speed', 'si_avg_speed', 'fc_avg_speed', 'sl_avg_speed', 'ch_avg_speed', 'cu_avg_speed',
               'fs_avg_speed', 'kn_avg_speed', 'st_avg_speed', 'sv_avg_speed']] = (
    df_pitch_arm_angle_zone_usage_spin_speed
      .groupby("player_id")[['ff_avg_speed', 'si_avg_speed', 'fc_avg_speed', 'sl_avg_speed', 'ch_avg_speed', 'cu_avg_speed',
               'fs_avg_speed', 'kn_avg_speed', 'st_avg_speed', 'sv_avg_speed']]
      .transform(lambda x: x.ffill().bfill())
)

Lets see that these values were indeed filled in

In [55]:
df_pitch_arm_angle_zone_usage_spin_speed[df_pitch_arm_angle_zone_usage_spin_speed['_merge']!='both']

,player_name,player_id,year,xwoba,xba,xslg,xiso,xobp,brl,brl_percent,...,si_avg_speed,fc_avg_speed,sl_avg_speed,ch_avg_speed,cu_avg_speed,fs_avg_speed,kn_avg_speed,st_avg_speed,sv_avg_speed,_merge
102,"Machado, Andrés",600921,2023,9.0,7.0,3.0,3.0,29.0,52.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
126,"Lynch IV, Daniel",663738,2023,21.0,17.0,16.0,19.0,42.0,65.0,54.0,...,93.8,0.0,86.8,86.0,82.6,0.0,0.0,0.0,0.0,left_only
162,"May, Trevor",543507,2023,33.0,47.0,62.0,70.0,4.0,87.0,81.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
219,"Mayza, Tim",641835,2023,82.0,35.0,81.0,92.0,64.0,95.0,94.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
234,"Ureña, José",570632,2023,6.0,17.0,5.0,2.0,12.0,56.0,12.0,...,96.0,0.0,87.0,88.7,0.0,0.0,0.0,0.0,0.0,left_only
236,"Armstrong, Shawn",542888,2023,98.0,88.0,92.0,89.0,98.0,97.0,95.0,...,93.6,90.5,0.0,0.0,0.0,0.0,0.0,85.2,83.6,left_only
292,"Fleming, Josh",676596,2023,11.0,3.0,16.0,39.0,10.0,79.0,83.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
315,"Alexander, Scott",518397,2023,72.0,19.0,69.0,88.0,58.0,92.0,94.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
337,"Almonte, Yency",622075,2023,41.0,39.0,68.0,77.0,14.0,83.0,68.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
357,"Speier, Gabe",642100,2023,80.0,52.0,63.0,70.0,79.0,68.0,34.0,...,94.4,0.0,83.2,0.0,0.0,0.0,0.0,0.0,0.0,left_only


Ok so the values were indeed filled in so we will proceed with dropping the unnesecary columns such as _merge and duplicate columns such as id and player name. We are also going to drop the knuckleball columns as this pitch is almost never seen in the MLB anymore

In [56]:
df_pitch_arm_angle_zone_usage_spin_speed = df_pitch_arm_angle_zone_usage_spin_speed.drop(
    columns = ['last_name, first_name','pitcher', '_merge',
               'kn_avg_speed', 'kn_avg_spin', 'n_kn'])

Great lets move on to the next dataset which is the df_batted dataset. This dataset will give us an idea of the batted ball outcomes for each pitcher in the dataset. For example, we can obtain the groundball or flyball rate which are the percentage of their batted balls that result in a groundball or flyball. This may give us an idea of a pitchers movement as well as their strategy to pitch to a flyball or groundball. We can also get other information such as the percentage of batted balls to each field

In [57]:
df_batted

,id,name,bbe,gb_rate,air_rate,fb_rate,ld_rate,pu_rate,pull_rate,straight_rate,oppo_rate,pull_gb_rate,straight_gb_rate,oppo_gb_rate,pull_air_rate,straight_air_rate,oppo_air_rate,year
0,666142,"Ragans, Cole",236,0.444915,0.555085,0.262712,0.241525,0.050847,0.338983,0.377119,0.283898,0.211864,0.182203,0.050847,0.127119,0.194915,0.233051,2023
1,669947,"Scholtens, Jesse",287,0.411150,0.588850,0.285714,0.243902,0.059233,0.383275,0.365854,0.250871,0.219512,0.153310,0.038328,0.163763,0.212544,0.212544,2023
2,607536,"Freeland, Kyle",540,0.409259,0.590741,0.296296,0.244444,0.050000,0.425926,0.370370,0.203704,0.246296,0.131481,0.031481,0.179630,0.238889,0.172222,2023
3,656234,"Bird, Jake",270,0.525926,0.474074,0.207407,0.229630,0.037037,0.407407,0.377778,0.214815,0.240741,0.233333,0.051852,0.166667,0.144444,0.162963,2023
4,694297,"Pfaadt, Brandon",298,0.325503,0.674497,0.338926,0.251678,0.083893,0.426174,0.352349,0.221477,0.154362,0.144295,0.026846,0.271812,0.208054,0.194631,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,527048,"Pérez, Martín",160,0.387500,0.612500,0.318750,0.193750,0.100000,0.418750,0.400000,0.181250,0.200000,0.156250,0.031250,0.218750,0.243750,0.150000,2025
345,694819,"Misiorowski, Jacob",152,0.348684,0.651316,0.282895,0.302632,0.065789,0.342105,0.381579,0.276316,0.164474,0.118421,0.065789,0.177632,0.263158,0.210526,2025
346,681857,"Olson, Reese",190,0.431579,0.568421,0.284211,0.242105,0.042105,0.426316,0.273684,0.300000,0.231579,0.147368,0.052632,0.194737,0.126316,0.247368,2025
347,656557,"Houck, Tanner",149,0.489933,0.510067,0.214765,0.261745,0.033557,0.416107,0.382550,0.201342,0.221477,0.221477,0.046980,0.194631,0.161074,0.154362,2025


We do not need all of these columns such as the groundball and flyball directions since we already have the pull and oppo rates for every batted ball so there is no need to include these as it will be double counting in a way. In addition, we can consolidate the flyball, linedrive and popup rates since those are already covered by the groundball and air rates

In [58]:
df_batted = df_batted.drop(columns = ['pull_gb_rate', 'straight_gb_rate', 'oppo_gb_rate', 'pull_air_rate',
                                      'straight_air_rate', 'oppo_air_rate', 'fb_rate', 'ld_rate', 'pu_rate'])

Great lets check how many NA values we have for this dataset

In [59]:
df_batted.isna().sum()

id               0
name             0
bbe              0
gb_rate          0
air_rate         0
pull_rate        0
straight_rate    0
oppo_rate        0
year             0
dtype: int64

Awesome no NA values here so lets merge in the dataframe with the rest and see if there are values that do not merge over

In [60]:
df_pitch_arm_angle_zone_usage_spin_speed_batted = df_pitch_arm_angle_zone_usage_spin_speed.merge(df_batted, left_on = ['player_id', 'year'],
                                                                                                 right_on = ['id', 'year'], how = 'left',
                                                                                                 indicator = True)

Lets see how many values merged over

In [61]:
df_pitch_arm_angle_zone_usage_spin_speed_batted['_merge'].value_counts()

_merge
both          1087
left_only        0
right_only       0
Name: count, dtype: int64

Awesome all of the values merged over so lets drop the columns we no longer need from the merge

In [62]:
df_pitch_arm_angle_zone_usage_spin_speed_batted = df_pitch_arm_angle_zone_usage_spin_speed_batted.drop(
    columns = ['_merge', 'id', 'name', 'bbe']
)

We can now move on to our final dataset which is df_bat_track. This can give us more information on the batted balls, specifically with regard to the quality of contact. For example the squared_up_per_swing metric is a measure of how many of a pitchers batted balls result in hard contact on only pitches that were swung at. There are several other similar metrics in the dataset as well

In [63]:
df_bat_track

,id,name,swings_competitive,percent_swings_competitive,contact,avg_bat_speed,hard_swing_rate,squared_up_per_bat_contact,squared_up_per_swing,blast_per_bat_contact,blast_per_swing,swing_length,swords,batter_run_value,whiffs,whiff_per_swing,batted_ball_events,batted_ball_event_per_swing,year
0,596295,"Gomber, Austin",320,0.935673,266,73.220150,0.343750,0.357143,0.296875,0.154135,0.128125,7.425720,2,-1.027199,54,0.168750,150,0.468750,2023
1,477132,"Kershaw, Clayton",216,0.931034,165,72.987322,0.342593,0.363636,0.277778,0.193939,0.148148,7.536529,3,-6.114433,51,0.236111,86,0.398148,2023
2,572971,"Keuchel, Dallas",231,0.946721,186,72.926831,0.320346,0.333333,0.268398,0.145161,0.116883,7.625250,4,-1.119203,45,0.194805,104,0.450216,2023
3,543135,"Eovaldi, Nathan",195,0.937500,151,72.786487,0.317949,0.337748,0.261538,0.152318,0.117949,7.418629,7,-6.490212,44,0.225641,71,0.364103,2023
4,471911,"Carrasco, Carlos",255,0.934066,194,72.710506,0.313725,0.443299,0.337255,0.247423,0.188235,7.564276,3,12.622678,61,0.239216,107,0.419608,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,681911,"Vesia, Alex",445,0.895372,310,71.060471,0.206742,0.229032,0.159551,0.103226,0.071910,7.142460,11,-18.054626,135,0.303371,113,0.253933,2025
203,672841,"Vargas, Carlos",462,0.905882,383,70.921646,0.188312,0.386423,0.320346,0.117493,0.097403,6.976493,10,-2.800287,79,0.170996,224,0.484848,2025
204,519242,"Sale, Chris",781,0.830851,578,70.860959,0.190781,0.285467,0.211268,0.102076,0.075544,7.372987,33,-21.742195,203,0.259923,257,0.329065,2025
205,676684,"Vest, Will",480,0.916031,363,70.833444,0.193750,0.316804,0.239583,0.126722,0.095833,7.005332,12,-10.316659,117,0.243750,171,0.356250,2025


Ok we do not need several of these columns. For example, we do not need the number of competitive swings, contact swings and batted ball events for each pitcher. I would rather keep the per swing metrics as it balances out the relief and starting pitchers since there is typically a significant difference in the number of innings between the two. By keeping it on a per swing basis, this can make the playing field even between the two

In [64]:
df_bat_track = df_bat_track.drop(
    columns = ['swings_competitive', 'swing_length', 'contact', 'avg_bat_speed', 'whiffs',
               'batted_ball_event_per_swing', 'batted_ball_events', 'blast_per_bat_contact',
               'squared_up_per_bat_contact', 'swords']
)

Ok great lets go ahead and see if we need to clean up any NA values

In [65]:
df_bat_track.isna().sum()

id                            0
name                          0
percent_swings_competitive    0
hard_swing_rate               0
squared_up_per_swing          0
blast_per_swing               0
batter_run_value              0
whiff_per_swing               0
year                          0
dtype: int64

Ok great there are no NA values in this dataframe so lets proceed with the merge

In [66]:
df_pitch_arm_angle_zone_usage_spin_speed_batted_track = df_pitch_arm_angle_zone_usage_spin_speed_batted.merge(
    df_bat_track, left_on = ['player_id', 'year'], right_on = ['id', 'year'], how = 'left', indicator = True
)

Lets see how many NA values there are in the dataset

In [67]:
df_pitch_arm_angle_zone_usage_spin_speed_batted_track['_merge'].value_counts()

_merge
both          643
left_only     444
right_only      0
Name: count, dtype: int64

Oof thats quite a few values that did not merge over so lets take a look to see if we can figure this out

In [68]:
df_pitch_arm_angle_zone_usage_spin_speed_batted_track[df_pitch_arm_angle_zone_usage_spin_speed_batted_track['_merge']!='both']

,player_name,player_id,year,xwoba,xba,xslg,xiso,xobp,brl,brl_percent,...,oppo_rate,id,name,percent_swings_competitive,hard_swing_rate,squared_up_per_swing,blast_per_swing,batter_run_value,whiff_per_swing,_merge
0,"McGough, Scott",543518,2023,44.0,58.0,40.0,36.0,46.0,46.0,13.0,...,0.213115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,"Contreras, Roansy",672710,2023,11.0,11.0,20.0,35.0,8.0,49.0,39.0,...,0.228972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
5,"Sandlin, Nick",680704,2023,57.0,82.0,45.0,21.0,70.0,60.0,16.0,...,0.278912,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
6,"VerHagen, Drew",572403,2023,57.0,78.0,67.0,51.0,45.0,62.0,46.0,...,0.248555,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
8,"Poche, Colin",621363,2023,98.0,96.0,96.0,91.0,94.0,83.0,83.0,...,0.237500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1077,"Topa, Justin",623437,2025,30.0,5.0,49.0,88.0,10.0,81.0,88.0,...,0.284264,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1078,"Ureña, José",570632,2025,1.0,2.0,1.0,1.0,1.0,39.0,1.0,...,0.254054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1079,"Hader, Josh",623352,2025,100.0,100.0,100.0,85.0,100.0,89.0,53.0,...,0.288288,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1082,"Eflin, Zach",621107,2025,26.0,14.0,9.0,12.0,66.0,41.0,30.0,...,0.229508,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


Seems to me it is several relief pitchers again, however, we cannot just assume a backfill and forward fill will be a good way to fill in the missing data since pitchers tend to perform differently from year to year if they are not a top tier pitcher. For now we are going to leave these values as NA and are not going to impute them because if we impute these then we may give several pitchers a false performance identity that isnt truly reflective of their actual performance and can throw off our comparisons. I would rather compare actual data to assumed data with this analysis, so lets just go ahead and drop the columns we no longer need from the merge

In [69]:
df_pitch_arm_angle_zone_usage_spin_speed_batted_track = df_pitch_arm_angle_zone_usage_spin_speed_batted_track.drop(
    columns = ['id', 'name', '_merge']
)

Ok now lets summarize the number of NA values we have from the merges that did not work and see how we can possibly deal with them. We are also going to rename the dataframe to a shorter name for simplicity

In [70]:
df = df_pitch_arm_angle_zone_usage_spin_speed_batted_track.copy()

In [71]:
df.isna().sum().loc[lambda x: x > 0].sort_values(ascending=False)

whiff_per_swing               444
batter_run_value              444
blast_per_swing               444
squared_up_per_swing          444
hard_swing_rate               444
percent_swings_competitive    444
runs_heart                    187
runs_shadow                   187
runs_chase                    187
runs_waste                    187
ball_angle                    133
pitch_hand                    133
n_fs                           11
fc_avg_speed                   11
n_ff                           11
n_si                           11
n_fc                           11
n_sl                           11
n_ch                           11
sv_avg_speed                   11
st_avg_speed                   11
fs_avg_speed                   11
cu_avg_speed                   11
ch_avg_speed                   11
sl_avg_speed                   11
si_avg_speed                   11
n_st                           11
ff_avg_speed                   11
sv_avg_spin                    11
st_avg_spin   

To be able to see all of the columns I am going to change the setting in pandas to see the maximum number of columns

In [72]:
pd.set_option('display.max_columns', None)

Lets quickly check one of the columns where we have exactly 11 NA values to see if we have 11 different players that have all of these columns NA

In [73]:
df[df['n_sv'].isna()]

,player_name,player_id,year,xwoba,xba,xslg,xiso,xobp,brl,brl_percent,hard_hit_percent,k_percent,bb_percent,whiff_percent,chase_percent,xera,pitch_hand,ball_angle,runs_heart,runs_shadow,runs_chase,runs_waste,n_ff,n_si,n_fc,n_sl,n_ch,n_cu,n_fs,n_st,n_sv,ff_avg_spin,si_avg_spin,fc_avg_spin,sl_avg_spin,ch_avg_spin,cu_avg_spin,fs_avg_spin,st_avg_spin,sv_avg_spin,ff_avg_speed,si_avg_speed,fc_avg_speed,sl_avg_speed,ch_avg_speed,cu_avg_speed,fs_avg_speed,st_avg_speed,sv_avg_speed,gb_rate,air_rate,pull_rate,straight_rate,oppo_rate,percent_swings_competitive,hard_swing_rate,squared_up_per_swing,blast_per_swing,batter_run_value,whiff_per_swing
102,"Machado, Andrés",600921,2023,9.0,7.0,3.0,3.0,29.0,52.0,6.0,47.0,32.0,80.0,34.0,83.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.440000,0.560000,0.353333,0.440000,0.206667,0.902954,0.228972,0.205607,0.11215,-4.522421,0.242991
162,"May, Trevor",543507,2023,33.0,47.0,62.0,70.0,4.0,87.0,81.0,71.0,23.0,3.0,31.0,13.0,33.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.323308,0.676692,0.428571,0.345865,0.225564,NaN,NaN,NaN,NaN,NaN,NaN
219,"Mayza, Tim",641835,2023,82.0,35.0,81.0,92.0,64.0,95.0,94.0,46.0,60.0,72.0,27.0,42.0,82.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.591837,0.408163,0.394558,0.333333,0.272109,NaN,NaN,NaN,NaN,NaN,NaN
292,"Fleming, Josh",676596,2023,11.0,3.0,16.0,39.0,10.0,79.0,83.0,1.0,1.0,46.0,4.0,32.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.615819,0.384181,0.401130,0.389831,0.209040,NaN,NaN,NaN,NaN,NaN,NaN
315,"Alexander, Scott",518397,2023,72.0,19.0,69.0,88.0,58.0,92.0,94.0,83.0,4.0,90.0,8.0,74.0,72.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.618182,0.381818,0.387879,0.430303,0.181818,NaN,NaN,NaN,NaN,NaN,NaN
337,"Almonte, Yency",622075,2023,41.0,39.0,68.0,77.0,14.0,83.0,68.0,93.0,51.0,15.0,79.0,25.0,41.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.427481,0.572519,0.465649,0.320611,0.213740,NaN,NaN,NaN,NaN,NaN,NaN
360,"Saucedo, Tayler",642048,2023,50.0,45.0,86.0,96.0,9.0,92.0,87.0,92.0,32.0,15.0,74.0,90.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.585185,0.407407,0.451852,0.311111,0.229630,NaN,NaN,NaN,NaN,NaN,NaN
534,"Pop, Zach",647315,2024,20.0,14.0,26.0,45.0,13.0,77.0,63.0,8.0,5.0,33.0,22.0,79.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.554839,0.445161,0.438710,0.354839,0.206452,NaN,NaN,NaN,NaN,NaN,NaN
660,"Ramírez, Yohan",670990,2024,24.0,30.0,56.0,72.0,4.0,88.0,70.0,83.0,38.0,47.0,44.0,8.0,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.455224,0.544776,0.410448,0.305970,0.283582,NaN,NaN,NaN,NaN,NaN,NaN
772,"Cavalli, Cade",676917,2025,39.0,20.0,56.0,76.0,30.0,95.0,95.0,38.0,18.0,71.0,72.0,95.0,39.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.550633,0.449367,0.398734,0.386076,0.215190,NaN,NaN,NaN,NaN,NaN,NaN


Ok so it seems like the players that have all of the pitch specific data NA are pitchers that only played for a year in the dataset and did not throw a lot of innings. Because there is no way to infer what types of pitches they threw, I am going to drop these players from the dataset as it is also a very small subset of the entire population

In [74]:
df = df.dropna(how ='all', subset = ['n_sl', 'n_ch', 'n_st', 'n_ff', 'n_cu', 'fs_avg_spin', 'ch_avg_spin', 'sl_avg_spin'])

Great lets check the number of NA values again to see if the above worked

In [75]:
df.isna().sum().loc[lambda x: x > 0].sort_values(ascending=False)

percent_swings_competitive    434
hard_swing_rate               434
squared_up_per_swing          434
blast_per_swing               434
batter_run_value              434
whiff_per_swing               434
runs_heart                    176
runs_shadow                   176
runs_chase                    176
runs_waste                    176
pitch_hand                    122
ball_angle                    122
dtype: int64

Great it looks like it worked. Lets now look into the contact based metrics where we still have 434 missing values in the dataset. I am going to infer that there is probably a subset of players where all of those values are missing since they all have the same number of missing values. If this is the case then we will likely drop those columns because I do not want to drop all of these players as it represents half of our dataset. In addition, I do not think imputing half the dataset would bring much value to the problem at hand. Lets view the players missing these metrics to see if we can find anything out

In [76]:
df_na = df[df['hard_swing_rate'].isna()]

In [77]:
df_na.head(40)

,player_name,player_id,year,xwoba,xba,xslg,xiso,xobp,brl,brl_percent,hard_hit_percent,k_percent,bb_percent,whiff_percent,chase_percent,xera,pitch_hand,ball_angle,runs_heart,runs_shadow,runs_chase,runs_waste,n_ff,n_si,n_fc,n_sl,n_ch,n_cu,n_fs,n_st,n_sv,ff_avg_spin,si_avg_spin,fc_avg_spin,sl_avg_spin,ch_avg_spin,cu_avg_spin,fs_avg_spin,st_avg_spin,sv_avg_spin,ff_avg_speed,si_avg_speed,fc_avg_speed,sl_avg_speed,ch_avg_speed,cu_avg_speed,fs_avg_speed,st_avg_speed,sv_avg_speed,gb_rate,air_rate,pull_rate,straight_rate,oppo_rate,percent_swings_competitive,hard_swing_rate,squared_up_per_swing,blast_per_swing,batter_run_value,whiff_per_swing
0,"McGough, Scott",543518,2023,44.0,58.0,40.0,36.0,46.0,46.0,13.0,1.0,84.0,26.0,87.0,81.0,44.0,R,46.3,2.386480,11.613077,-5.527187,-6.940499,50.6,0.5,0.0,9.3,0.0,0.0,39.6,0.0,0.0,2332.0,2378.0,0.0,2544.0,0.0,0.0,1185.0,0.0,0.0,93.5,94.1,0.0,86.6,0.0,0.0,85.7,0.0,0.0,0.491803,0.508197,0.453552,0.333333,0.213115,NaN,NaN,NaN,NaN,NaN,NaN
1,"Contreras, Roansy",672710,2023,11.0,11.0,20.0,35.0,8.0,49.0,39.0,10.0,16.0,20.0,54.0,50.0,11.0,R,50.0,5.672986,-4.185587,-9.694340,-4.180542,40.2,0.0,0.0,43.5,3.9,11.7,0.0,0.8,0.0,2353.0,0.0,0.0,2551.0,2040.0,2628.0,0.0,2583.0,0.0,94.3,0.0,0.0,83.6,89.4,76.8,0.0,82.3,0.0,0.392523,0.607477,0.425234,0.345794,0.228972,NaN,NaN,NaN,NaN,NaN,NaN
5,"Sandlin, Nick",680704,2023,57.0,82.0,45.0,21.0,70.0,60.0,16.0,43.0,80.0,26.0,87.0,48.0,57.0,R,5.5,4.658019,11.598140,-5.248079,-4.818995,26.5,19.5,0.0,49.7,0.0,0.0,4.4,0.0,0.0,2320.0,2157.0,0.0,2601.0,0.0,0.0,1515.0,0.0,0.0,92.1,92.4,0.0,79.1,0.0,0.0,85.9,0.0,0.0,0.421769,0.578231,0.476190,0.244898,0.278912,NaN,NaN,NaN,NaN,NaN,NaN
6,"VerHagen, Drew",572403,2023,57.0,78.0,67.0,51.0,45.0,62.0,46.0,68.0,41.0,31.0,80.0,42.0,57.0,R,48.7,0.911304,10.708558,-4.515886,-6.993444,27.3,17.9,12.6,0.0,11.2,0.0,0.0,31.1,0.0,2299.0,2270.0,2342.0,0.0,1572.0,0.0,0.0,2320.0,0.0,93.4,92.6,88.2,0.0,88.5,0.0,0.0,81.3,0.0,0.410405,0.589595,0.462428,0.289017,0.248555,NaN,NaN,NaN,NaN,NaN,NaN
8,"Poche, Colin",621363,2023,98.0,96.0,96.0,91.0,94.0,83.0,83.0,93.0,60.0,29.0,71.0,78.0,98.0,NaN,NaN,NaN,NaN,NaN,NaN,65.0,0.0,0.0,35.0,0.0,0.0,0.0,0.0,0.0,2249.0,0.0,0.0,2448.0,0.0,0.0,0.0,0.0,0.0,92.5,0.0,0.0,84.6,0.0,0.0,0.0,0.0,0.0,0.362500,0.637500,0.437500,0.325000,0.237500,NaN,NaN,NaN,NaN,NaN,NaN
13,"Lauer, Eric",641778,2023,1.0,1.0,1.0,1.0,2.0,44.0,1.0,8.0,30.0,12.0,20.0,14.0,1.0,L,38.6,NaN,NaN,NaN,NaN,43.1,0.1,35.4,8.2,0.7,12.4,0.0,0.0,0.0,2184.0,2220.0,2210.0,2258.0,1498.0,2341.0,0.0,0.0,0.0,90.8,90.4,86.7,82.6,82.8,75.3,0.0,0.0,0.0,0.250000,0.750000,0.472222,0.312500,0.215278,NaN,NaN,NaN,NaN,NaN,NaN
18,"Moreta, Dauri",664294,2023,94.0,98.0,88.0,67.0,93.0,74.0,46.0,87.0,95.0,26.0,92.0,6.0,94.0,NaN,NaN,6.653706,9.991651,-2.159488,-6.405262,26.4,0.0,0.0,64.8,8.8,0.0,0.0,0.0,0.0,2318.0,0.0,0.0,1991.0,1731.0,0.0,0.0,0.0,0.0,95.2,0.0,0.0,84.6,85.8,0.0,0.0,0.0,0.0,0.404412,0.595588,0.492647,0.338235,0.169118,NaN,NaN,NaN,NaN,NaN,NaN
20,"Schreiber, John",670167,2023,53.0,75.0,75.0,71.0,25.0,87.0,73.0,47.0,71.0,7.0,38.0,17.0,53.0,R,11.8,NaN,NaN,NaN,NaN,28.6,25.9,0.2,0.0,7.8,0.0,0.0,37.5,0.0,2206.0,2218.0,2181.0,0.0,1662.0,0.0,0.0,2614.0,0.0,93.2,92.7,90.7,0.0,88.1,0.0,0.0,80.9,0.0,0.434426,0.565574,0.327869,0.352459,0.319672,NaN,NaN,NaN,NaN,NaN,NaN
21,"McClanahan, Shane",663556,2023,53.0,53.0,44.0,43.0,61.0,25.0,20.0,25.0,68.0,45.0,92.0,84.0,53.0,L,43.3,11.191466,16.512095,-6.996996,-5.262598,41.8,0.1,0.0,15.3,25.8,17.0,0.0,0.0,0.0,2242.0,2320.0,0.0,2256.0,1604.0,2532.0,0.0,0.0,0.0,96.8,95.9,0.0,89.3,86.9,83.5,0.0,0.0,0.0,0.446254,0.553746,0.342020,0.387622,0.270358,NaN,NaN,NaN,NaN,NaN,NaN
23,"Rogers, Taylor",573124,2023,76.0,88.0,79.0,66.0,57.0,68.0,22.0,8.0,89.0,11.0,56.0,35.0,76.0,L,29.1,NaN,NaN,NaN,NaN,0.7,40.8,2.0,0.0,0.0,0.0,0.0,56.5,0.0,1915.0,2121.0,2186.0,0.0,0.0,0.0,0.0,2730.0,0.0,93.4,93.6,88.5,0.0,0.0,0.0,0.0,79.4,0.0,0.435484,0.564516,0.387097,0.411290,0.201613,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
df_na.tail(40)

,player_name,player_id,year,xwoba,xba,xslg,xiso,xobp,brl,brl_percent,hard_hit_percent,k_percent,bb_percent,whiff_percent,chase_percent,xera,pitch_hand,ball_angle,runs_heart,runs_shadow,runs_chase,runs_waste,n_ff,n_si,n_fc,n_sl,n_ch,n_cu,n_fs,n_st,n_sv,ff_avg_spin,si_avg_spin,fc_avg_spin,sl_avg_spin,ch_avg_spin,cu_avg_spin,fs_avg_spin,st_avg_spin,sv_avg_spin,ff_avg_speed,si_avg_speed,fc_avg_speed,sl_avg_speed,ch_avg_speed,cu_avg_speed,fs_avg_speed,st_avg_speed,sv_avg_speed,gb_rate,air_rate,pull_rate,straight_rate,oppo_rate,percent_swings_competitive,hard_swing_rate,squared_up_per_swing,blast_per_swing,batter_run_value,whiff_per_swing
980,"Muñoz, Andrés",662253,2025,93.0,95.0,98.0,96.0,69.0,89.0,79.0,55.0,96.0,11.0,98.0,82.0,93.0,R,29.9,12.046152,11.438117,-2.354216,-4.935111,35.1,12.4,0.0,50.4,2.1,0.0,0.0,0.0,0.0,2279.0,2048.0,0.0,2356.0,1496.0,0.0,0.0,0.0,0.0,98.4,97.8,0.0,86.1,91.6,0.0,0.0,0.0,0.0,0.514493,0.485507,0.369565,0.355072,0.275362,NaN,NaN,NaN,NaN,NaN,NaN
988,"Pérez, Martín",527048,2025,7.0,33.0,5.0,1.0,23.0,39.0,1.0,59.0,26.0,22.0,17.0,11.0,7.0,L,47.0,NaN,NaN,NaN,NaN,3.5,31.2,28.2,0.0,28.8,8.3,0.0,0.0,0.0,2071.0,1987.0,2220.0,0.0,1650.0,2443.0,0.0,0.0,0.0,89.7,89.5,86.0,0.0,82.3,76.4,0.0,0.0,0.0,0.387500,0.612500,0.418750,0.400000,0.181250,NaN,NaN,NaN,NaN,NaN,NaN
990,"Bachar, Lake",669199,2025,26.0,66.0,21.0,9.0,43.0,46.0,14.0,43.0,71.0,19.0,84.0,78.0,26.0,R,36.2,14.177363,6.288367,-7.771834,-6.994687,37.2,0.0,0.0,23.3,0.0,0.0,12.2,27.4,0.0,2717.0,0.0,0.0,2657.0,0.0,0.0,1299.0,2871.0,0.0,94.8,0.0,0.0,89.2,0.0,0.0,84.0,86.5,0.0,0.351064,0.648936,0.414894,0.340426,0.244681,NaN,NaN,NaN,NaN,NaN,NaN
991,"Hernández, Carlos",672578,2025,5.0,8.0,9.0,17.0,2.0,59.0,7.0,64.0,24.0,13.0,59.0,34.0,5.0,R,41.5,NaN,NaN,NaN,NaN,55.9,0.1,0.0,23.7,0.0,9.2,11.2,0.0,0.0,2437.0,2195.0,0.0,2068.0,0.0,2375.0,1347.0,0.0,0.0,99.1,98.0,0.0,88.6,0.0,84.1,89.0,0.0,0.0,0.342857,0.657143,0.385714,0.378571,0.235714,NaN,NaN,NaN,NaN,NaN,NaN
997,"Junis, Jakob",596001,2025,45.0,23.0,43.0,61.0,47.0,76.0,82.0,64.0,30.0,74.0,27.0,37.0,45.0,R,27.5,7.824775,2.271578,-1.490420,-3.707218,8.6,26.9,0.0,44.5,20.0,0.0,0.0,0.0,0.0,2144.0,2122.0,0.0,2480.0,1313.0,0.0,0.0,0.0,0.0,91.2,91.1,0.0,83.0,86.4,0.0,0.0,0.0,0.0,0.421320,0.578680,0.436548,0.365482,0.197970,NaN,NaN,NaN,NaN,NaN,NaN
999,"Allard, Kolby",663465,2025,79.0,47.0,57.0,61.0,82.0,76.0,87.0,83.0,6.0,92.0,10.0,25.0,79.0,L,46.0,10.593477,0.352893,-3.561830,-2.726926,38.9,8.6,19.3,0.0,20.0,13.2,0.0,0.0,0.0,2029.0,2005.0,2039.0,0.0,1547.0,1882.0,0.0,0.0,0.0,90.1,90.6,84.9,0.0,81.4,72.1,0.0,0.0,0.0,0.382775,0.617225,0.344498,0.411483,0.244019,NaN,NaN,NaN,NaN,NaN,NaN
1000,"Dobbins, Hunter",690928,2025,51.0,46.0,36.0,32.0,67.0,72.0,75.0,40.0,14.0,74.0,22.0,43.0,51.0,NaN,NaN,NaN,NaN,NaN,NaN,40.5,1.4,0.0,27.3,0.0,12.0,8.7,10.1,0.0,2371.0,2242.0,0.0,2568.0,0.0,2482.0,1462.0,2535.0,0.0,95.5,95.0,0.0,87.6,0.0,78.7,90.5,81.2,0.0,0.484375,0.515625,0.333333,0.343750,0.322917,NaN,NaN,NaN,NaN,NaN,NaN
1002,"Iglesias, Raisel",628452,2025,85.0,82.0,64.0,46.0,91.0,62.0,45.0,88.0,81.0,76.0,88.0,91.0,85.0,NaN,NaN,9.079582,9.463917,-4.584794,-2.269457,40.4,21.7,0.0,9.9,28.1,0.0,0.0,0.0,0.0,2344.0,2332.0,0.0,2634.0,1999.0,0.0,0.0,0.0,0.0,94.9,94.7,0.0,83.7,88.8,0.0,0.0,0.0,0.0,0.320000,0.680000,0.360000,0.354286,0.285714,NaN,NaN,NaN,NaN,NaN,NaN
1003,"Armstrong, Shawn",542888,2025,88.0,89.0,86.0,73.0,88.0,72.0,69.0,91.0,76.0,68.0,43.0,8.0,88.0,R,41.0,14.563812,17.726698,-7.565613,-4.913932,28.9,23.6,24.1,0.0,0.0,0.0,0.0,23.3,0.1,2344.0,2136.0,2468.0,0.0,0.0,0.0,0.0,2615.0,2683.0,93.5,94.1,90.8,0.0,0.0,0.0,0.0,85.1,84.6,0.358696,0.641304,0.320652,0.385870,0.293478,NaN,NaN,NaN,NaN,NaN,NaN
1004,"Buttó, José",676130,2025,25.0,33.0,34.0,39.0,12.0,51.0,30.0,56.0,33.0,9.0,76.0,62.0,25.0,R,37.8,0.082689,8.424963,-2.823808,-6.756983,30.9,17.0,0.0,30.0,15.2,0.0,0.0,7.0,0.0,2411.0,2176.0,0.0,2644.0,1900.0,0.0,0.0,2561.0,0.0,95.2,94.7,0.0,86.5,88.4,0.0,0.0,83.6,0.0,0.481675,0.518325,0.397906,0.366492,0.235602,NaN,NaN,NaN,NaN,NaN,NaN


Ok so looking at around 80 of the values we can see that these pitchers are all indeed missing every single one of the contact metrics. As mentioned above, since about half of the rows are NA for the per swing columns, we are going to drop those instead of imputing half of the dataframe. Although these metrics would bring valuable comparisons between players, it doesnt make much sense to impute half of the players in the dataset

In [79]:
df = df.drop(
    columns = ['percent_swings_competitive', 'hard_swing_rate', 'squared_up_per_swing',        
                'blast_per_swing','batter_run_value', 'whiff_per_swing']
)

Ok lets take a look at the remaining columns that have NA values

In [80]:
df.isna().sum().loc[lambda x: x > 0].sort_values(ascending=False)

runs_heart     176
runs_shadow    176
runs_chase     176
runs_waste     176
pitch_hand     122
ball_angle     122
dtype: int64

It seems like several of the zone specific values are NA and they all have the same number of values so we may have a similar situation where the same players have all of these values missing. Lets investigate this

In [81]:
df_na = df[df['runs_heart'].isna()]

In [82]:
df_na.head(40)

,player_name,player_id,year,xwoba,xba,xslg,xiso,xobp,brl,brl_percent,hard_hit_percent,k_percent,bb_percent,whiff_percent,chase_percent,xera,pitch_hand,ball_angle,runs_heart,runs_shadow,runs_chase,runs_waste,n_ff,n_si,n_fc,n_sl,n_ch,n_cu,n_fs,n_st,n_sv,ff_avg_spin,si_avg_spin,fc_avg_spin,sl_avg_spin,ch_avg_spin,cu_avg_spin,fs_avg_spin,st_avg_spin,sv_avg_spin,ff_avg_speed,si_avg_speed,fc_avg_speed,sl_avg_speed,ch_avg_speed,cu_avg_speed,fs_avg_speed,st_avg_speed,sv_avg_speed,gb_rate,air_rate,pull_rate,straight_rate,oppo_rate
8,"Poche, Colin",621363,2023,98.0,96.0,96.0,91.0,94.0,83.0,83.0,93.0,60.0,29.0,71.0,78.0,98.0,NaN,NaN,NaN,NaN,NaN,NaN,65.0,0.0,0.0,35.0,0.0,0.0,0.0,0.0,0.0,2249.0,0.0,0.0,2448.0,0.0,0.0,0.0,0.0,0.0,92.5,0.0,0.0,84.6,0.0,0.0,0.0,0.0,0.0,0.362500,0.637500,0.437500,0.325000,0.237500
13,"Lauer, Eric",641778,2023,1.0,1.0,1.0,1.0,2.0,44.0,1.0,8.0,30.0,12.0,20.0,14.0,1.0,L,38.6,NaN,NaN,NaN,NaN,43.1,0.1,35.4,8.2,0.7,12.4,0.0,0.0,0.0,2184.0,2220.0,2210.0,2258.0,1498.0,2341.0,0.0,0.0,0.0,90.8,90.4,86.7,82.6,82.8,75.3,0.0,0.0,0.0,0.250000,0.750000,0.472222,0.312500,0.215278
20,"Schreiber, John",670167,2023,53.0,75.0,75.0,71.0,25.0,87.0,73.0,47.0,71.0,7.0,38.0,17.0,53.0,R,11.8,NaN,NaN,NaN,NaN,28.6,25.9,0.2,0.0,7.8,0.0,0.0,37.5,0.0,2206.0,2218.0,2181.0,0.0,1662.0,0.0,0.0,2614.0,0.0,93.2,92.7,90.7,0.0,88.1,0.0,0.0,80.9,0.0,0.434426,0.565574,0.327869,0.352459,0.319672
23,"Rogers, Taylor",573124,2023,76.0,88.0,79.0,66.0,57.0,68.0,22.0,8.0,89.0,11.0,56.0,35.0,76.0,L,29.1,NaN,NaN,NaN,NaN,0.7,40.8,2.0,0.0,0.0,0.0,0.0,56.5,0.0,1915.0,2121.0,2186.0,0.0,0.0,0.0,0.0,2730.0,0.0,93.4,93.6,88.5,0.0,0.0,0.0,0.0,79.4,0.0,0.435484,0.564516,0.387097,0.411290,0.201613
33,"Nelson, Kyle",669459,2023,69.0,65.0,40.0,30.0,89.0,60.0,23.0,46.0,81.0,85.0,83.0,96.0,69.0,NaN,NaN,NaN,NaN,NaN,NaN,33.1,0.0,7.6,59.2,0.0,0.0,0.0,0.0,0.0,2217.0,0.0,2566.0,2687.0,0.0,0.0,0.0,0.0,0.0,92.0,0.0,88.1,85.4,0.0,0.0,0.0,0.0,0.0,0.394904,0.605096,0.464968,0.305732,0.229299
46,"Thompson, Mason",666168,2023,33.0,30.0,57.0,75.0,20.0,83.0,86.0,72.0,16.0,37.0,15.0,14.0,33.0,NaN,NaN,NaN,NaN,NaN,NaN,6.9,56.0,0.0,28.5,0.2,8.4,0.0,0.0,0.0,2230.0,2247.0,0.0,2278.0,1851.0,2398.0,0.0,0.0,0.0,94.5,94.7,0.0,84.7,86.3,80.5,0.0,0.0,0.0,0.505882,0.494118,0.317647,0.400000,0.282353
53,"Silseth, Chase",681217,2023,44.0,65.0,59.0,53.0,34.0,62.0,16.0,25.0,64.0,11.0,63.0,23.0,44.0,NaN,NaN,NaN,NaN,NaN,NaN,35.6,14.2,8.4,2.7,0.0,2.4,16.7,20.0,0.0,2401.0,2334.0,2478.0,2684.0,0.0,2804.0,1132.0,2777.0,0.0,95.0,94.3,90.3,85.3,0.0,79.4,87.2,81.6,0.0,0.489051,0.510949,0.467153,0.372263,0.160584
56,"Avila, Pedro",658648,2023,55.0,58.0,83.0,89.0,18.0,95.0,92.0,29.0,58.0,12.0,70.0,38.0,55.0,R,45.0,NaN,NaN,NaN,NaN,30.4,23.0,0.0,0.0,25.6,20.1,0.0,0.9,0.0,2238.0,2273.0,0.0,0.0,1645.0,2687.0,0.0,2782.0,0.0,94.1,93.3,0.0,0.0,83.8,77.4,0.0,79.7,0.0,0.602941,0.397059,0.389706,0.382353,0.227941
59,"Bernardino, Brennan",657514,2023,53.0,49.0,69.0,75.0,37.0,83.0,73.0,87.0,75.0,50.0,39.0,1.0,53.0,NaN,NaN,NaN,NaN,NaN,NaN,0.2,55.7,0.0,0.0,1.4,42.7,0.0,0.0,0.0,1520.0,2098.0,0.0,0.0,1670.0,2616.0,0.0,0.0,0.0,89.7,90.9,0.0,0.0,82.4,77.6,0.0,0.0,0.0,0.492537,0.507463,0.328358,0.417910,0.253731
71,"Bido, Osvaldo",674370,2023,26.0,11.0,66.0,89.0,4.0,92.0,92.0,63.0,29.0,42.0,25.0,29.0,26.0,R,33.9,NaN,NaN,NaN,NaN,27.8,23.6,2.4,36.3,9.9,0.0,0.0,0.0,0.0,2387.0,2202.0,2131.0,2157.0,2113.0,0.0,0.0,0.0,0.0,94.7,94.6,85.8,84.5,89.1,0.0,0.0,0.0,0.0,0.358491,0.641509,0.364780,0.345912,0.289308


In [83]:
df_na.tail(40)

,player_name,player_id,year,xwoba,xba,xslg,xiso,xobp,brl,brl_percent,hard_hit_percent,k_percent,bb_percent,whiff_percent,chase_percent,xera,pitch_hand,ball_angle,runs_heart,runs_shadow,runs_chase,runs_waste,n_ff,n_si,n_fc,n_sl,n_ch,n_cu,n_fs,n_st,n_sv,ff_avg_spin,si_avg_spin,fc_avg_spin,sl_avg_spin,ch_avg_spin,cu_avg_spin,fs_avg_spin,st_avg_spin,sv_avg_spin,ff_avg_speed,si_avg_speed,fc_avg_speed,sl_avg_speed,ch_avg_speed,cu_avg_speed,fs_avg_speed,st_avg_speed,sv_avg_speed,gb_rate,air_rate,pull_rate,straight_rate,oppo_rate
815,"Legumina, Casey",668984,2025,58.0,77.0,64.0,49.0,47.0,76.0,43.0,22.0,70.0,8.0,44.0,45.0,58.0,NaN,NaN,NaN,NaN,NaN,NaN,49.2,22.8,0.0,0.0,10.7,0.0,0.0,17.3,0.0,2433.0,2288.0,0.0,0.0,1583.0,0.0,0.0,2688.0,0.0,94.3,93.9,0.0,0.0,87.7,0.0,0.0,81.1,0.0,0.394161,0.605839,0.299270,0.364964,0.335766
821,"Hill, Tim",657612,2025,77.0,47.0,61.0,68.0,79.0,55.0,58.0,14.0,2.0,85.0,2.0,96.0,77.0,L,-19.6,NaN,NaN,NaN,NaN,13.5,82.2,0.0,4.3,0.0,0.0,0.0,0.0,0.0,2114.0,2058.0,0.0,2033.0,0.0,0.0,0.0,0.0,0.0,89.9,88.4,0.0,84.7,0.0,0.0,0.0,0.0,0.0,0.641860,0.358140,0.306977,0.427907,0.265116
825,"Jansen, Kenley",445276,2025,61.0,81.0,28.0,9.0,84.0,67.0,41.0,17.0,63.0,46.0,54.0,51.0,61.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,9.2,81.4,6.1,0.0,0.0,0.0,3.3,0.0,0.0,2336.0,2602.0,2457.0,0.0,0.0,0.0,2513.0,0.0,0.0,93.2,92.8,83.9,0.0,0.0,0.0,82.0,0.0,0.286624,0.713376,0.420382,0.337580,0.242038
830,"Houck, Tanner",656557,2025,1.0,1.0,5.0,14.0,1.0,76.0,53.0,9.0,6.0,41.0,20.0,57.0,1.0,R,22.1,NaN,NaN,NaN,NaN,0.1,32.1,1.5,0.0,0.0,0.0,24.9,41.4,0.0,2274.0,2191.0,2347.0,0.0,0.0,0.0,1836.0,2534.0,0.0,95.1,93.5,87.1,0.0,0.0,0.0,88.4,83.2,0.0,0.489933,0.510067,0.416107,0.382550,0.201342
847,"Jackson, Luke",592426,2025,34.0,51.0,50.0,51.0,7.0,85.0,76.0,21.0,13.0,3.0,27.0,30.0,34.0,NaN,NaN,NaN,NaN,NaN,NaN,31.6,0.0,1.0,52.0,0.0,15.4,0.0,0.0,0.0,2295.0,0.0,2294.0,2264.0,0.0,2403.0,0.0,0.0,0.0,94.7,0.0,90.8,87.3,0.0,84.6,0.0,0.0,0.0,0.526667,0.473333,0.500000,0.300000,0.200000
862,"Scott, Tanner",656945,2025,38.0,36.0,33.0,35.0,49.0,67.0,39.0,34.0,71.0,65.0,79.0,99.0,38.0,L,35.2,NaN,NaN,NaN,NaN,52.7,0.0,0.0,47.2,0.1,0.0,0.0,0.0,0.0,2550.0,0.0,0.0,2633.0,2382.0,0.0,0.0,0.0,0.0,96.5,0.0,0.0,88.9,93.7,0.0,0.0,0.0,0.0,0.397436,0.602564,0.397436,0.358974,0.243590
863,"Speier, Gabe",642100,2025,98.0,97.0,94.0,85.0,99.0,72.0,39.0,48.0,96.0,97.0,88.0,98.0,98.0,NaN,NaN,NaN,NaN,NaN,NaN,44.7,29.5,0.0,25.7,0.0,0.0,0.0,0.0,0.0,2234.0,2088.0,0.0,2185.0,0.0,0.0,0.0,0.0,0.0,95.0,94.4,0.0,83.2,0.0,0.0,0.0,0.0,0.0,0.400000,0.600000,0.324138,0.331034,0.344828
879,"Myers, Tobias",668964,2025,31.0,16.0,26.0,42.0,39.0,81.0,76.0,16.0,13.0,71.0,18.0,35.0,31.0,R,60.5,NaN,NaN,NaN,NaN,45.2,0.0,19.5,16.8,1.5,1.5,15.6,0.0,0.0,2280.0,0.0,2158.0,2128.0,1801.0,2317.0,1529.0,0.0,0.0,93.5,0.0,87.7,84.5,80.1,76.4,83.1,0.0,0.0,0.369697,0.630303,0.345455,0.381818,0.272727
880,"Bernardino, Brennan",657514,2025,78.0,83.0,91.0,90.0,39.0,81.0,66.0,85.0,23.0,6.0,9.0,21.0,78.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,44.8,12.4,4.1,12.5,26.3,0.0,0.0,0.0,0.0,2174.0,2315.0,2370.0,1601.0,2688.0,0.0,0.0,0.0,0.0,90.7,86.5,83.9,79.8,79.2,0.0,0.0,0.0,0.503311,0.496689,0.311258,0.410596,0.278146
889,"Walter, Brandon",687888,2025,82.0,54.0,57.0,57.0,94.0,81.0,67.0,64.0,64.0,100.0,44.0,94.0,82.0,NaN,NaN,NaN,NaN,NaN,NaN,19.3,12.5,27.4,0.0,18.3,0.0,0.0,22.5,0.0,2179.0,2105.0,2303.0,0.0,2077.0,0.0,0.0,2555.0,0.0,91.8,91.5,88.2,0.0,80.9,0.0,0.0,79.5,0.0,0.444444,0.555556,0.496732,0.274510,0.228758


Again it looks as if this is for pitchers that are starters with a low number of innings pitched or relief pitchers who did not qualify with enough innings or pitches to register run values in each area of the zone. In this case since I would like to keep these players in the analysis for the sake of comparison, I am going to leave these values NA. It is not a reasonable assumption to impute these pitchers with the mean value or with 0 because their true value could be drastically different from this. When we compute the similarity of these players, we are going to only compare them on the values that are not NA. We are going to do the same with pitcher angle as it is difficult to infer the median value for a pitchers arm angle. This few number of NA values also does not warrant dropping the column entirely as it is not a significant number of the total rows that are missing. Although this may make it so that these players will almost never have a high score in this process, I would rather err on that side then to assume a certain performance standard and tell a front office that a certain player is much better than they appear by imputing these values

Now lets go ahead and create a manual dictionary for the pitcher hand for the pitchers that are missing this value. We will first explore the exact pitchers that are missing the value then create the dictionary

In [84]:
df_no_hand = df[df['pitch_hand'].isna()]

Now list the players that are missing the pitch hand data

In [85]:
df_no_hand['player_name'].unique()

array(['Poche, Colin', 'Moreta, Dauri', 'Nelson, Kyle', 'Thompson, Mason',
       'Silseth, Chase', 'Englert, Mason', 'Bernardino, Brennan',
       'Nardi, Andrew', 'Iglesias, Raisel', 'Hand, Brad',
       'Leone, Dominic', 'Cosgrove, Tom', 'Smith, Will',
       'Marinaccio, Ron', 'Murphy, Chris', 'Pérez, Cionel',
       'Gallegos, Giovanny', 'Cueto, Johnny', 'López, Jorge',
       'Adon, Joan', 'Woodford, Jake', 'Young, Alex', 'Davidson, Tucker',
       'Vasquez, Andrew', 'Martinez, Adrián', 'Brasier, Ryan',
       'Ruiz, José', 'Gonzales, Marco', 'Hentges, Sam',
       'Hernandez, Jose E.', 'Gott, Trevor', 'Loup, Aaron',
       'Fulmer, Michael', 'Ryu, Hyun Jin', 'Harvey, Hunter',
       'Middleton, Keynan', 'Soriano, George', 'Honeywell, Brent',
       'Romano, Jordan', 'Coulombe, Danny', 'Graterol, Brusdar',
       'Weems, Jordan', 'Sborz, Josh', 'Phillips, Evan', 'Peguero, Elvis',
       'Chafin, Andrew', 'Holderman, Colin', 'Alzolay, Adbert',
       'Speier, Gabe', 'Ramírez, Eras

Create the dictionary that will map the pitch hand to the player name

In [86]:
hand_dict = {'Poche, Colin':'L', 'Moreta, Dauri':'R', 'Nelson, Kyle':'L', 'Thompson, Mason':'R',
       'Silseth, Chase':'R', 'Englert, Mason':'R', 'Bernardino, Brennan':'L',
       'Nardi, Andrew':'L', 'Iglesias, Raisel':'R', 'Hand, Brad':'L',
       'Leone, Dominic':'R', 'Cosgrove, Tom':'L', 'Smith, Will':'L',
       'Marinaccio, Ron':'R', 'Murphy, Chris':'L', 'Pérez, Cionel':'L',
       'Gallegos, Giovanny':'R', 'Cueto, Johnny':'R', 'López, Jorge':'R',
       'Adon, Joan':'R', 'Woodford, Jake':'R', 'Young, Alex':'L', 'Davidson, Tucker':'L',
       'Vasquez, Andrew':'L', 'Martinez, Adrián':'R', 'Brasier, Ryan':'R',
       'Ruiz, José':'R', 'Gonzales, Marco':'L', 'Hentges, Sam':'L',
       'Hernandez, Jose E.':'L', 'Gott, Trevor':'R', 'Loup, Aaron':'L',
       'Fulmer, Michael':'R', 'Ryu, Hyun Jin':'L', 'Harvey, Hunter':'R',
       'Middleton, Keynan':'R', 'Soriano, George':'R', 'Honeywell, Brent':'R',
       'Romano, Jordan':'R', 'Coulombe, Danny':'L', 'Graterol, Brusdar':'R',
       'Weems, Jordan':'R', 'Sborz, Josh':'R', 'Phillips, Evan':'R', 'Peguero, Elvis':'R',
       'Chafin, Andrew':'L', 'Holderman, Colin':'R', 'Alzolay, Adbert':'R',
       'Speier, Gabe':'L', 'Ramírez, Erasmo':'R', 'Bard, Daniel':'R', 'Moore, Matt':'L',
       'Jansen, Kenley':'R', 'Martinez, Seth':'R', 'Wilson, Justin':'L',
       'Green, Chad':'R', 'Jackson, Luke':'R', 'Garcia, Robert':'L', 'Smith, Burch':'R',
       'Nicolas, Kyle':'R', 'McArthur, James':'R', 'Turnbull, Spencer':'R',
       'Bender, Anthony':'R', 'Miller, Tyson':'R', 'Garrett, Reed':'R',
       'Hudson, Bryan':'L', 'Rodríguez, Randy':'R', 'McFarland, T.J.':'L',
       'Wicks, Jordan':'L', 'Helsley, Ryan':'R', 'King, John':'L', 'Mantiply, Joe':'L',
       'Dubin, Shawn':'R', 'Miller, Shelby':'R', 'Winn, Keaton':'R', 'Rainey, Tanner':'R',
       'Boyle, Joe':'R', 'Thielbar, Caleb':'L', 'Slaten, Justin':'R', 'Kelly, Zack':'R',
       'Anderson, Justin':'R', 'Svanson, Matt':'R', 'Gibson, Cade':'L',
       'Montgomery, Mason':'L', 'Legumina, Casey':'R', 'Henry, Cole':'R',
       'Walter, Brandon':'L', 'Young, Brandon':'R', 'Palencia, Daniel':'R',
       'Jobe, Jackson':'R', 'Hanifee, Brenan':'R', 'Fluharty, Mason':'L',
       'Dobbins, Hunter':'R', 'Festa, Matt':'R', 'Waldrep, Hurston':'R',
       'Trivino, Lou':'R', 'Gilbert, Tyler':'L'}

df['pitch_hand'] = (
    df['pitch_hand']
      .fillna(df['player_name'].map(hand_dict))
)

Great lets check to see if there are any remaining NA values for pitch hand

In [87]:
df.isna().sum().loc[lambda x: x > 0].sort_values(ascending=False)

runs_heart     176
runs_shadow    176
runs_chase     176
runs_waste     176
ball_angle     122
dtype: int64

Great now we can proceed with the function to compare pitchers

Ok so the code below will have a weight for each of the years. I wanted to apply a recency weighting to give more emphasis towards recent performance. Each of the metrics below will be multiplied by the weight for each of the years and then will be summed up to give the total weighted value for each player across the 3 years of data. If a player is not present in all 3 years then the weighting is adjusted to reflect the one or two seasons they were present. After this weighting is completed we will then group by each player ID to get a single row for each of the players in the dataset. We do not include any categorical variables in the actual weighting as there is no way to actually compile a weighted version of these variables

In [88]:
weights = {2025: 0.6, 2024: 0.3, 2023: 0.1}
df["w"] = df["year"].map(weights)

# keep only years we want
df = df[df["w"].notna()].copy()

# renormalize weights per player
df["w_norm"] = df["w"] / df.groupby("player_id")["w"].transform("sum")

metrics = ['xwoba', 'xba', 'xslg', 'xiso',
       'xobp', 'brl', 'brl_percent',
       'hard_hit_percent', 'k_percent', 'bb_percent', 'whiff_percent',
       'chase_percent', 'xera', 'ball_angle', 'runs_heart',
       'runs_shadow', 'runs_chase', 'runs_waste', 'n_ff', 'n_si', 'n_fc',
       'n_sl', 'n_ch', 'n_cu', 'n_fs', 'n_st', 'n_sv', 'ff_avg_spin',
       'si_avg_spin', 'fc_avg_spin', 'sl_avg_spin', 'ch_avg_spin',
       'cu_avg_spin', 'fs_avg_spin', 'st_avg_spin',
       'sv_avg_spin', 'ff_avg_speed', 'si_avg_speed', 'fc_avg_speed',
       'sl_avg_speed', 'ch_avg_speed', 'cu_avg_speed', 'fs_avg_speed',
       'st_avg_speed', 'sv_avg_speed', 'gb_rate', 'air_rate',
       'pull_rate', 'straight_rate','oppo_rate']

# weighted values
for m in metrics:
    df[m] = df[m] * df["w_norm"]

# make sure last() means most recent year for side/position
df = df.sort_values(["player_id", "year"])

agg = (
    df.groupby("player_id", as_index=False)
      .agg(
          player_name=("player_name", "first"),
          seasons=("year", "nunique"),
          pitch_hand=("pitch_hand", "last"),
          **{m: (m, lambda s: s.sum(min_count=1)) for m in metrics}
      )
)

In [89]:
agg

,player_id,player_name,seasons,pitch_hand,xwoba,xba,xslg,xiso,xobp,brl,brl_percent,hard_hit_percent,k_percent,bb_percent,whiff_percent,chase_percent,xera,ball_angle,runs_heart,runs_shadow,runs_chase,runs_waste,n_ff,n_si,n_fc,n_sl,n_ch,n_cu,n_fs,n_st,n_sv,ff_avg_spin,si_avg_spin,fc_avg_spin,sl_avg_spin,ch_avg_spin,cu_avg_spin,fs_avg_spin,st_avg_spin,sv_avg_spin,ff_avg_speed,si_avg_speed,fc_avg_speed,sl_avg_speed,ch_avg_speed,cu_avg_speed,fs_avg_speed,st_avg_speed,sv_avg_speed,gb_rate,air_rate,pull_rate,straight_rate,oppo_rate
0,425794,"Wainwright, Adam",1,R,1.000000,1.000000,1.000000,1.000000,2.000000,7.000000,7.000000,51.0,1.000000,47.000000,1.000000,2.000000,1.000000,45.500000,-15.120591,0.077559,-18.339039,-7.240569,10.000000,31.400000,22.600000,0.300000,5.000000,30.800000,0.000000,0.000000,0.0,2159.000000,2167.0,2322.000000,2458.000000,1655.000000,2647.000000,0.000000,0.0,0.000000,85.700000,86.900000,82.900000,75.600000,81.40,71.500000,0.000000,0.000000,0.0,0.410853,0.589147,0.410853,0.405685,0.183463
1,425844,"Greinke, Zack",1,R,18.000000,5.000000,10.000000,21.000000,44.000000,14.000000,53.000000,51.0,9.000000,98.000000,2.000000,38.000000,18.000000,43.800000,1.351353,15.968263,-14.094608,-7.590029,26.700000,15.600000,7.000000,20.000000,16.500000,14.100000,0.000000,0.000000,0.0,2248.000000,2205.0,2437.000000,2464.000000,1755.000000,2408.000000,0.000000,0.0,0.000000,89.500000,89.800000,85.200000,79.200000,86.50,72.400000,0.000000,0.000000,0.0,0.437100,0.562900,0.353945,0.373134,0.272921
2,434378,"Verlander, Justin",3,R,46.700000,40.900000,41.000000,42.400000,55.300000,24.000000,56.500000,77.9,30.300000,58.300000,35.700000,42.100000,46.700000,55.040000,6.447052,14.171707,-13.365178,-7.391333,46.730000,0.270000,0.000000,22.300000,8.600000,17.330000,0.000000,4.740000,0.0,2421.100000,2217.6,0.000000,2483.800000,1753.500000,2725.900000,0.000000,1603.2,0.000000,93.820000,83.340000,0.000000,86.960000,84.54,78.200000,0.000000,48.420000,0.0,0.335646,0.664354,0.417127,0.340133,0.242740
3,445276,"Jansen, Kenley",2,R,67.000000,82.666667,41.666667,22.666667,82.333333,62.666667,28.666667,33.0,70.000000,41.333333,56.666667,37.666667,67.000000,NaN,NaN,NaN,NaN,NaN,0.000000,9.200000,81.400000,6.100000,0.000000,0.000000,0.000000,3.300000,0.0,0.000000,2336.0,2602.000000,2457.000000,0.000000,0.000000,0.000000,2513.0,0.000000,0.000000,93.200000,92.800000,83.900000,0.00,0.000000,0.000000,82.000000,0.0,0.294787,0.705213,0.398773,0.348510,0.252717
4,445926,"Chavez, Jesse",1,R,36.000000,17.000000,34.000000,52.000000,40.000000,72.000000,76.000000,76.0,32.000000,63.000000,2.000000,2.000000,36.000000,21.600000,3.388383,10.435699,-9.191858,-4.352279,0.000000,26.900000,51.100000,1.600000,13.400000,7.000000,0.000000,0.000000,0.0,0.000000,1988.0,2201.000000,2412.000000,1882.000000,2558.000000,0.000000,0.0,0.000000,0.000000,90.700000,88.800000,81.400000,85.20,75.700000,0.000000,0.000000,0.0,0.433862,0.566138,0.296296,0.407407,0.296296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
588,801403,"Dollander, Chase",1,R,11.000000,33.000000,18.000000,14.000000,8.000000,24.000000,6.000000,50.0,20.000000,11.000000,28.000000,4.000000,11.000000,25.800000,2.447653,8.515818,-16.277921,-11.705784,48.700000,9.900000,12.400000,0.000000,7.800000,21.300000,0.000000,0.000000,0.0,2433.000000,2411.0,2572.000000,0.000000,2072.000000,2808.000000,0.000000,0.0,0.000000,97.900000,97.200000,89.700000,0.000000,89.30,79.200000,0.000000,0.000000,0.0,0.424342,0.575658,0.351974,0.384868,0.263158
589,805673,"Matthews, Zebby",1,R,19.000000,16.000000,19.000000,24.000000,41.000000,39.000000,21.000000,64.0,68.000000,71.000000,68.000000,74.000000,19.000000,39.600000,-5.076406,5.181478,-7.005425,-3.865976,41.100000,4.400000,13.000000,25.100000,10.600000,5.800000,0.000000,0.000000,0.0,2331.000000,2317.0,2494.000000,2419.000000,

Lets change the name to be a more readable format of firstname lastname by using common pandas string methods

In [90]:
agg['player_name_lst'] = agg['player_name'].str.split(',')

In [91]:
agg['player_name_final'] = agg['player_name_lst'].apply(lambda x:x[1] + " " + x[0])
agg = agg.drop(columns = ['player_name_lst', 'player_name'])

Lets set the index as the player name to make it easier to view different parts of the data

In [92]:
agg['player_name_final'] = agg['player_name_final'].str.strip()
agg = agg.set_index('player_name_final')

Ok great lets now scale the numerical variables so that no one variable dominates the rest

In [93]:
X = agg[metrics].copy()

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_scaled = pd.DataFrame(X_scaled, columns=metrics, index=agg.index)

In [94]:
metadata_cols = ["pitch_hand"]

X_scaled_full = pd.concat(
    [agg[metadata_cols], X_scaled],
    axis=1
)

In [95]:
X_scaled_full

,pitch_hand,xwoba,xba,xslg,xiso,xobp,brl,brl_percent,hard_hit_percent,k_percent,bb_percent,whiff_percent,chase_percent,xera,ball_angle,runs_heart,runs_shadow,runs_chase,runs_waste,n_ff,n_si,n_fc,n_sl,n_ch,n_cu,n_fs,n_st,n_sv,ff_avg_spin,si_avg_spin,fc_avg_spin,sl_avg_spin,ch_avg_spin,cu_avg_spin,fs_avg_spin,st_avg_spin,sv_avg_spin,ff_avg_speed,si_avg_speed,fc_avg_speed,sl_avg_speed,ch_avg_speed,cu_avg_speed,fs_avg_speed,st_avg_speed,sv_avg_speed,gb_rate,air_rate,pull_rate,straight_rate,oppo_rate
player_name_final,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Adam Wainwright,R,-1.732125,-1.750675,-1.744036,-1.749473,-1.693698,-1.689296,-1.581842,0.056840,-1.726049,-0.014152,-1.722165,-1.649177,-1.732125,0.598588,-3.076239,-1.526908,-1.795287,-0.407024,-1.219553,0.937435,1.215881,-1.012630,-0.471403,2.231323,-0.368065,-0.605861,-0.147886,-0.093581,0.532940,1.026888,0.643741,0.460228,0.963151,-0.455444,-0.839142,-0.205080,-0.280031,0.389681,0.940974,0.335902,0.503589,0.661888,-0.467800,-0.845178,-0.206386,-0.209818,0.209991,0.560149,1.297509,-1.820623
Zack Greinke,R,-1.099788,-1.603741,-1.412402,-1.009711,-0.117823,-1.418412,0.166830,0.056840,-1.433967,1.866758,-1.686399,-0.347901,-1.099788,0.465736,-0.542342,0.642183,-0.963610,-0.524134,-0.273022,-0.003213,-0.071360,0.230349,0.614628,0.597945,-0.368065,-0.605861,-0.147886,0.109223,0.576400,1.126212,0.649503,0.584128,0.769865,-0.455444,-0.839142,-0.205080,-0.059058,0.468361,0.994391,0.434760,0.638725,0.685216,-0.467800,-0.845178,-0.206386,0.151891,-0.151722,-0.807050,0.258450,0.726087
Justin Verlander,R,-0.032253,-0.285012,-0.270109,-0.218166,0.306162,-1.031434,0.299882,1.055974,-0.656300,0.402599,-0.481070,-0.199700,-0.032253,1.344123,0.241535,0.396952,-0.820681,-0.457548,0.862248,-0.915879,-0.648968,0.375468,-0.131428,0.913862,-0.368065,-0.200707,-0.147886,0.503665,0.590811,-0.978601,0.668520,0.582270,1.026960,-0.455444,0.453771,-0.205080,0.192153,0.293095,-0.984327,0.647853,0.586791,0.835553,-0.467800,0.388212,-0.206386,-1.246220,1.246405,0.710889,-0.795005,-0.133111
Kenley Jansen,R,0.722832,1.249220,-0.245543,-0.948064,1.320475,0.464879,-0.758192,-0.611725,0.793155,-0.223142,0.268833,-0.359950,0.722832,NaN,NaN,NaN,NaN,NaN,-1.786338,-0.384234,6.067788,-0.646677,-0.943590,-0.781135,-0.368065,-0.323791,-0.147886,-5.013288,0.726226,1.268721,0.642780,-1.590325,-1.177549,-0.455444,1.187486,-0.205080,-5.263552,0.560606,1.170896,0.563824,-1.653286,-1.191404,-0.467800,1.243587,-0.206386,-1.809291,1.809482,0.269945,-0.527601,0.150911
Jesse Chavez,R,-0.430254,-1.162940,-0.528046,0.136919,-0.267907,0.826058,1.041167,0.985404,-0.594233,0.575937,-1.686399,-1.649177,-0.430254,-1.269158,-0.228984,-0.113015,-0.002939,0.560891,-1.786338,0.669529,3.567571,-0.930606,0.321872,-0.096485,-0.368065,-0.605861,-0.147886,-5.013288,0.328216,0.922381,0.599560,0.741482,0.891174,-0.455444,-0.839142,-0.205080,-5.263552,0.492779,1.077998,0.495173,0.604279,0.770752,-0.467800,-0.845178,-0.206386,0.107272,-0.107103,-2.192030,1.352499,1.391533
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Chase Dollander,R,-1.360162,-0.575206,-1.117617,-1.268628,-1.468573,-1.031434,-1.619857,0.019698,-1.032355,-1.341853,-0.756471,-1.576884,-1.360162,-0.940935,-0.373697,-0.375080,-1.391420,-1.903395,0.973905,-0.342560,0.374224,-1.031559,-0.206978,1.302156,-0.368065,-0.605861,-0.147886,0.530781,0.812004,1.242810,-1.717021,0.976892,1.093356,-0.455444,-0.839142,-0.205080,0.429408,0.669129,1.098900,-1.740110,0.712917,0.861473,-0.467800,-0.845178,-0.206386,-0.023925,0.024096,-0.854400,0.633020,0.448148
Zebby Matthews,R,-1.062591,-1.199674,-1.080769,-0.898747,-0.230386,-0.450968,-1.049638,0.539693,0.720135,0.870982,0.674186,0.953375,-1.062591,0.137513,-1.531131,-0.830220,0.425483,0.723860,0.543148,-0.670001,0.423733,0.552135,0.057447,-0.213854,-0.368065,-0.605861,-0.14788

Lets save the cleaned pitcher dataset to its own file so that we can use it in the final function

In [96]:
X_scaled_full.to_csv('../data/cleaned_player_data/cleaned_pitchers.csv')